Input and Output
================

<img src="img/dognap.png" title="poor dog" style="background-color:white;float:right;margin-left:2em;" />

We've mentioned that Haskell is a purely functional language. Whereas in
imperative languages you usually get things done by giving the computer
a series of steps to execute, functional programming is more of defining
what stuff is. In Haskell, a function can't change some state, like
changing the contents of a variable (when a function changes state, we
say that the function has *side-effects*). The only thing a function can
do in Haskell is give us back some result based on the parameters we
gave it. If a function is called two times with the same parameters, it
has to return the same result. While this may seem a bit limiting when
you're coming from an imperative world, we've seen that it's actually
really cool. In an imperative language, you have no guarantee that a
simple function that should just crunch some numbers won't burn down
your house, kidnap your dog and scratch your car with a potato while
crunching those numbers. For instance, when we were making a binary
search tree, we didn't insert an element into a tree by modifying some
tree in place. Our function for inserting into a binary search tree
actually returned a new tree, because it can't change the old one.

While functions being unable to change state is good because it helps us
reason about our programs, there's one problem with that. If a function
can't change anything in the world, how is it supposed to tell us what
it calculated? In order to tell us what it calculated, it has to change
the state of an output device (usually the state of the screen), which
then emits photons that travel to our brain and change the state of our
mind, man.

Do not despair, all is not lost. It turns out that Haskell actually has
a really clever system for dealing with functions that have side-effects
that neatly separates the part of our program that is pure and the part
of our program that is impure, which does all the dirty work like
talking to the keyboard and the screen. With those two parts separated,
we can still reason about our pure program and take advantage of all the
things that purity offers, like laziness, robustness and modularity
while efficiently communicating with the outside world.

> __Jupyter Note:__ We'll turn off the [automatic linting for IHaskell](https://github.com/IHaskell/IHaskell/wiki#opt-no-lint) first.

In [1]:
:opt no-lint

> __Jupyter Note:__ About the following functions.
>
> `withStdin` takes a string and runs an [`IO`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IO) action with that string as the standard input.
This is for simulating an interactive command line environment for programs. Inspired by [Test.Main.withStdin](http://hackage.haskell.org/package/main-tester-0.2.0.1/docs/src/Test.Main.html#withStdin).
> The original book instructs the reader to run interactive programs from a shell command line, but those instructions don't work in a Jupyter notebook environment
> because we want to be able to run the notebook from beginning to end without waiting for user interaction. The programs in this chapter can still be run
> interactively in the notebook, but if a program doesn't terminate, you will have to <kbd>⭮</kbd> Restart the kernel.
>
> `catchPrint` catches an exception and prints it to standard output. We use this because Jupyter won't show us the output of a program if the program throws an exception. We want to see both the output and the exception.

In [70]:
import System.IO
import GHC.IO.Handle
import Control.Exception
import System.Directory

withStdin :: String -> IO a -> IO a
withStdin s action = do
    writeFile "/tmp/stdin.txt" s
    finally
        (withFile "/tmp/stdin.txt" ReadWriteMode
            (\h -> do
                  stdin' <- hDuplicate stdin
                  hDuplicateTo h stdin
                  finally action (hDuplicateTo stdin' stdin)            
            )
        )
        (removeFile "/tmp/stdin.txt")

catchPrint = flip catch p where
    p :: SomeException -> IO ()
    p = print

Line 19: Use handle
Found:
flip catch
Why not:
handle

Hello, world!
-------------

<img src="img/helloworld.png" title="HELLO!" style="background-color:white;float:left;margin-right:2em;" />

Up until now, we've always loaded our functions into GHCI to test them
out and play with them. We've also explored the standard library
functions that way. But now, after eight or so chapters, we're finally
going to write our first *real* Haskell program! Yay! And sure enough,
we're going to do the good old `"hello, world"` schtick.

In [71]:
main = putStrLn "hello, world"

In [72]:
main

hello, world

We just defined a name called `main` and in it we call a function called
[`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) with the parameter `"hello, world"`. Looks pretty much run of the
mill, but it isn't, as we'll see in just a few moments.

Let's examine what we wrote. First, let's look at the type of the
function [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn).

In [73]:
:t putStrLn

putStrLn :: String -> IO ()

In [74]:
:t putStrLn "hello, world"

putStrLn "hello, world" :: IO ()

We can read the type of [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) like this: [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) takes a string and
returns an *I/O action* that has a result type of `()` (i.e. the empty
tuple, also know as unit). An I/O action is something that, when
performed, will carry out an action with a side-effect (that's usually
either reading from the input or printing stuff to the screen) and will
also contain some kind of return value inside it. Printing a string to
the terminal doesn't really have any kind of meaningful return value, so
a dummy value of `()` is used.

> The empty tuple is a value of `()` and it also has a type of `()`.

So, when will an I/O action be performed? Well, this is where `main` comes
in. An I/O action will be performed when we give it a name of `main` and
then run our program.

Having your whole program be just one I/O action seems kind of limiting.
That's why we can use *do* syntax to glue together several I/O actions
into one. Take a look at the following example:

In [75]:
main = do
    putStrLn "Hello, what's your name?"
    name <- getLine
    putStrLn ("Hey " ++ name ++ ", you rock!")

Ah, interesting, new syntax! And this reads pretty much like an
imperative program. If you compile it and try it out, it will probably
behave just like you expect it to. Notice that we said *do* and then we
laid out a series of steps, like we would in an imperative program. Each
of these steps is an I/O action. By putting them together with *do*
syntax, we glued them into one I/O action. The action that we got has a
type of `IO ()`, because that's the type of the last I/O action inside.

Because of that, `main` always has a type signature of
`main :: IO <something>`,
where `<something>` is some concrete type. By convention, we
don't usually specify a type declaration for `main`.

An interesting thing that we haven't met before is the third line, which
states `name <- getLine`. It looks like it reads a line from the input
and stores it into a variable called `name`. Does it really? Well, let's
examine the type of [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine).

In [76]:
:t getLine

getLine :: IO String

<img src="img/luggage.png" title="luggage" style="background-color:white;float:left;margin-right:2em;" />

Aha, o-kay. __[`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine)__ is an I/O action that contains a result type of
[`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String). That makes sense, because it will wait for the user to input
something at the terminal and then that something will be represented as
a string. So what's up with `name <- getLine` then? You can read that
piece of code like this: *perform the I/O action [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) and then bind
its result value to `name`*. [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) has a type of `IO String`, so `name` will
have a type of [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String). You can think of an I/O action as a box with
little feet that will go out into the real world and do something there
(like write some graffiti on a wall) and maybe bring back some data.
Once it's fetched that data for you, the only way to open the box and
get the data inside it is to use the `<-` construct. And if we're taking
data out of an I/O action, we can only take it out when we're inside
another I/O action. This is how Haskell manages to neatly separate the
pure and impure parts of our code. [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) is in a sense impure because
its result value is not guaranteed to be the same when performed twice.
That's why it's sort of *tainted* with the [`IO`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IO) type constructor and we
can only get that data out in I/O code. And because I/O code is tainted
too, any computation that depends on tainted I/O data will have a
tainted result.

When I say *tainted*, I don't mean tainted in such a way that we can
never use the result contained in an I/O action ever again in pure code.
No, we temporarily *un-taint* the data inside an I/O action when we bind
it to a name. When we do `name <- getLine`, `name` is just a normal string,
because it represents what's inside the box. We can have a really
complicated function that, say, takes your name (a normal string) as a
parameter and tells you your fortune and your whole life's future based
on your name. We can do this:

<pre><code>main = do
    putStrLn "Hello, what's your name?"
    name <- getLine
    putStrLn $ "Read this carefully, because this is your future: " ++ tellFortune name</code></pre>

and `tellFortune` (or any of the functions it passes `name` to) doesn't have
to know anything about I/O, it's just a normal `String -> String`
function!

Take a look at this piece of code. Is it valid?

<pre><code>nameTag = "Hello, my name is " ++ getLine</code></pre>

If you said no, go eat a cookie. If you said yes, drink a bowl of molten
lava. Just kidding, don't! The reason that this doesn't work is that [`++`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:-43--43-)
requires both its parameters to be lists over the same type. The left
parameter has a type of [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String) (or `[Char]` if you will), whilst [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine)
has a type of `IO String`. You can't concatenate a string and an I/O
action. We first have to get the result out of the I/O action to get a
value of type [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String) and the only way to do that is to say something
like `name <- getLine` inside some other I/O action. If we want to deal
with impure data, we have to do it in an impure environment. So the
taint of impurity spreads around much like the undead scourge and it's
in our best interest to keep the I/O parts of our code as small as
possible.

Every I/O action that gets performed has a result encapsulated within
it. That's why our previous example program could also have been written
like this:

<pre><code>main = do
    foo <- putStrLn "Hello, what's your name?"
    name <- getLine
    putStrLn ("Hey " ++ name ++ ", you rock!")</code></pre>

However, `foo` would just have a value of `()`, so doing that would be kind
of moot. Notice that we didn't bind the last [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) to anything.
That's because in a *do* block, *the last action cannot be bound to a
name* like the first two were. We'll see exactly why that is so a bit
later when we venture off into the world of monads. For now, you can
think of it in the way that the *do* block automatically extracts the
value from the last action and binds it to its own result.

Except for the last line, every line in a *do* block that doesn't bind
can also be written with a bind. So `putStrLn "BLAH"` can be written as
`_ <- putStrLn "BLAH"`. But that's useless, so we leave out the `<-` for I/O
actions that don't contain an important result, like `putStrLn <something>`.

Beginners sometimes think that doing

<pre><code>name = getLine</code></pre>

will read from the input and then bind the value of that to `name`. Well,
it won't, all this does is give the [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) I/O action a different name
called, well, `name`. Remember, to get the value out of an I/O action, you
have to perform it inside another I/O action by binding it to a name
with `<-`.

I/O actions will only be performed when they are given a name of `main` or
when they're inside a bigger I/O action that we composed with a *do*
block. We can also use a *do* block to glue together a few I/O actions
and then we can use that I/O action in another *do* block and so on.
Either way, they'll be performed only if they eventually fall into `main`.

Oh, right, there's also one more case when I/O actions will be
performed. When we type out an I/O action in GHCI and press return, it
will be performed.

In [77]:
putStrLn "HEEY"

HEEY

Even when we just punch out a number or call a function in GHCI and
press return, it will evaluate it (as much as it needs) and then call
[`show`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:show) on it and then it will print that string to the terminal using
[`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) implicitly.

Remember *let* bindings? If you don't, refresh your memory on them by
reading [this section](http://learnyouahaskell.com/syntax-in-functions#let-it-be). They have to be
in the form of `let <bindings> in <expression>`, where `<bindings>` are
names to be given to expressions and `<expression>` is the expression that
is to be evaluated that sees them. We also said that in list
comprehensions, the *in* part isn't needed. Well, you can use them in
*do* blocks pretty much like you use them in list comprehensions. Check
this out:

In [78]:
import Data.Char

main = do
    putStrLn "What's your first name?"
    firstName <- getLine
    putStrLn "What's your last name?"
    lastName <- getLine
    let bigFirstName = map toUpper firstName
        bigLastName = map toUpper lastName
    putStrLn $ "hey " ++ bigFirstName ++ " " ++ bigLastName ++ ", how are you?"

In [79]:
withStdin (unlines ["John", "Brown"]) main

What's your first name?
What's your last name?
hey JOHN BROWN, how are you?

See how the I/O actions in the *do* block are lined up? Also notice how
the *let* is lined up with the I/O actions and the names of the *let*
are lined up with each other? That's good practice, because indentation
is important in Haskell. Now, we did `map toUpper firstName`, which turns
something like `"John"` into a much cooler string like `"JOHN"`. We bound
that uppercased string to a name and then used it in a string later on
that we printed to the terminal.

You may be wondering when to use `<-` and when to use *let* bindings?
Well, remember, `<-` is (for now) for performing I/O actions and binding
their results to names. map toUpper firstName, however, isn't an I/O
action. It's a pure expression in Haskell. So use `<-` when you want to
bind results of I/O actions to names and you can use *let* bindings to
bind pure expressions to names. Had we done something like
`let firstName = getLine`, we would have just called the [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) I/O action a different
name and we'd still have to run it through a `<-` to perform it.

Now we're going to make a program that continuously reads a line and
prints out the same line with the words reversed. The program's
execution will stop when we input a blank line. This is the program:

In [80]:
main = do
    line <- getLine
    if null line
        then return ()
        else do
            putStrLn $ reverseWords line
            main

reverseWords :: String -> String
reverseWords = unwords . map reverse . words

To get a feel of what it does, you can run it before we go over the
code.

In [81]:
withStdin "hey there man" $ catchPrint main

yeh ereht nam
<stdin>: hGetLine: end of file

First, let's take a look at the `reverseWords` function. It's just a
normal function that takes a string like "hey there man" and then calls
[`words`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:words) with it to produce a list of words like `["hey","there","man"]`.
Then we map [`reverse`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:reverse) on the list, getting `["yeh","ereht","nam"]` and then
we put that back into one string by using [`unwords`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:unwords) and the final result
is `"yeh ereht nam"`. See how we used function composition here. Without
function composition, we'd have to write something like
`reverseWords st = unwords (map reverse (words st))`.

What about `main`? First, we get a line from the terminal by performing
[`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) call that line `line`. And now, we have a conditional expression.
Remember that in Haskell, every *if* must have a corresponding *else*
because every expression has to have some sort of value. We make the
*if* so that when a condition is true (in our case, the line that we
entered is blank), we perform one I/O action and when it isn't, the I/O
action under the *else* is performed. That's why in an I/O *do* block,
*if*s have to have a form of
`if <condition> then <I/O action> else <I/O action>`.

Let's first take a look at what happens under the *else* clause.
Because, we have to have exactly one I/O action after the *else*, we use
a *do* block to glue together two I/O actions into one. You could also
write that part out as:

<pre><code>        else (do
            putStrLn $ reverseWords line
            main)</code></pre>

This makes it more explicit that the *do* block can be viewed as one I/O
action, but it's uglier. Anyway, inside the *do* block, we call
`reverseWords` on the line that we got from [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) and then print that
out to the terminal. After that, we just perform `main`. It's called
recursively and that's okay, because `main` is itself an I/O action. So in
a sense, we go back to the start of the program.

Now what happens when `null line` holds true? What's after the *then* is
performed in that case. If we look up, we'll see that it says
`then return ()`. If you've done imperative languages like C, Java or Python,
you're probably thinking that you know what this [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) does and chances
are you've already skipped this really long paragraph. Well, here's the
thing: *the [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) in Haskell is really nothing like the [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) in most
other languages!* It has the same name, which confuses a lot of people,
but in reality it's quite different. In imperative languages, [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return)
usually ends the execution of a method or subroutine and makes it report
some sort of value to whoever called it. In Haskell (in I/O actions
specifically), it makes an I/O action out of a pure value. If you think
about the box analogy from before, it takes a value and wraps it up in a
box. The resulting I/O action doesn't actually do anything, it just has
that value encapsulated as its result. So in an I/O context,
`return "haha"` will have a type of `IO String`. What's the point of just
transforming a pure value into an I/O action that doesn't do anything?
Why taint our program with [`IO`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IO) more than it has to be? Well, we needed
some I/O action to carry out in the case of an empty input line. That's
why we just made a bogus I/O action that doesn't do anything by writing
`return ()`.

Using [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) doesn't cause the I/O *do* block to end in execution or
anything like that. For instance, this program will quite happily carry
out all the way to the last line:

In [82]:
main = do
    return ()
    return "HAHAHA"
    line <- getLine
    return "BLAH BLAH BLAH"
    return 4
    putStrLn line

Line 2: Redundant return
Found:
return ()
Why not:
Line 3: Redundant return
Found:
return "HAHAHA"
Why not:
Line 5: Redundant return
Found:
return "BLAH BLAH BLAH"
Why not:
Line 6: Redundant return
Found:
return 4
Why not:

All these [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return)s do is that they make I/O actions that don't really do
anything except have an encapsulated result and that result is thrown
away because it isn't bound to a name. We can use [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) in combination
with `<-` to bind stuff to names.

In [83]:
main = do
    a <- return "hell"
    b <- return "yeah!"
    putStrLn $ a ++ " " ++ b

Line 2: Use let
Found:
a <- return "hell"
Why not:
let a = "hell"Line 3: Use let
Found:
b <- return "yeah!"
Why not:
let b = "yeah!"

So you see, [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) is sort of the opposite to `<-`. While [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) takes a
value and wraps it up in a box, `<-` takes a box (and performs it) and
takes the value out of it, binding it to a name. But doing this is kind
of redundant, especially since you can use *let* bindings in *do* blocks
to bind to names, like so:

In [84]:
main = do
    let a = "hell"
        b = "yeah"
    putStrLn $ a ++ " " ++ b

When dealing with I/O *do* blocks, we mostly use [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) either because
we need to create an I/O action that doesn't do anything or because we
don't want the I/O action that's made up from a *do* block to have the
result value of its last action, but we want it to have a different
result value, so we use [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) to make an I/O action that always has our
desired result contained and we put it at the end.

> A *do* block can also have just one I/O action. In that case, it's the
> same as just writing the I/O action. Some people would prefer writing
> `then do return ()` in this case because the *else* also has a *do*.

Before we move on to files, let's take a look at some functions that are
useful when dealing with I/O.

__[`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr)__ is much like [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) in that it takes a string as a parameter
and returns an I/O action that will print that string to the terminal,
only [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr) doesn't jump into a new line after printing out the string
while [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) does.

In [85]:
main = do   putStr "Hey, "
            putStr "I'm "
            putStrLn "Andy!"

In [86]:
main

Hey, I'm Andy!

Its type signature is `putStr :: String -> IO ()`, so the result
encapsulated within the resulting I/O action is the unit. A dud value,
so it doesn't make sense to bind it.

__[`putChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putChar)__ takes a character and returns an I/O action that will print it
out to the terminal.

In [87]:
main = do   putChar 't'
            putChar 'e'
            putChar 'h'

In [88]:
main

teh

[`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr) is actually defined recursively with the help of [`putChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putChar). The
edge condition of [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr) is the empty string, so if we're printing an
empty string, just return an I/O action that does nothing by using
`return ()`. If it's not empty, then print the first character of the
string by doing [`putChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putChar) and then print of them using [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr).

<pre><code>putStr :: String -> IO ()
putStr [] = return ()
putStr (x:xs) = do
    putChar x
    putStr xs</code></pre>

See how we can use recursion in I/O, just like we can use it in pure
code. Just like in pure code, we define the edge case and then think
what the result actually is. It's an action that first outputs the first
character and then outputs the rest of the string.

__[`print`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:print)__ takes a value of any type that's an instance of [`Show`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Show) (meaning that
we know how to represent it as a string), calls [`show`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:show) with that value to
stringify it and then outputs that string to the terminal. Basically,
it's just `putStrLn . show`. It first runs [`show`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:show) on a value and then feeds
that to [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn), which returns an I/O action that will print out our
value.

In [89]:
main = do   print True
            print 2
            print "haha"
            print 3.2
            print [3,4,3]

In [90]:
main

True
2
"haha"
3.2
[3,4,3]

As you can see, it's a very handy function. Remember how we talked about
how I/O actions are performed only when they fall into `main` or when we
try to evaluate them in the GHCI prompt? When we type out a value (like
`3` or `[1,2,3]`) and press the return key, GHCI actually uses [`print`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:print) on that
value to display it on our terminal!

In [91]:
3

3

In [92]:
print 3

3

In [93]:
map (++"!") ["hey","ho","woo"]

["hey!","ho!","woo!"]

In [94]:
print (map (++"!") ["hey","ho","woo"])

["hey!","ho!","woo!"]

When we want to print out strings, we usually use [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) because we
don't want the quotes around them, but for printing out values of other
types to the terminal, [`print`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:print) is used the most.

__[`getChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getChar)__ is an I/O action that reads a character from the input. Thus,
its type signature is `getChar :: IO Char`, because the result contained
within the I/O action is a [`Char`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Char). Note that due to buffering, reading of
the characters won't actually happen until the user mashes the return
key.

In [95]:
main = do
    c <- getChar
    if c /= ' '
        then do
            putChar c
            main
        else return ()

Line 3: Use when
Found:
if c /= ' ' then
    do putChar c
       main
else
    return ()
Why not:
Control.Monad.when (c /= ' ')
  $ do putChar c
       main

This program looks like it should read a character and then check if
it's a space. If it is, halt execution and if it isn't, print it to the
terminal and then do the same thing all over again. Well, it kind of
does, only not in the way you might expect. Check this out:

In [96]:
withStdin "hello sir" main

hello

The second line is the input. We input `hello sir` and then press return.
Due to buffering, the execution of the program will begin only when
after we've hit return and not after every inputted character. But once
we press return, it acts on what we've been putting in so far. Try
playing with this program to get a feel for it!

The __[`when`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:when)__ function is found in `Control.Monad` (to get access to it, do
`import Control.Monad`). It's interesting because in a *do* block it looks
like a control flow statement, but it's actually a normal function. It
takes a boolean value and an I/O action if that boolean value is [`True`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:True),
it returns the same I/O action that we supplied to it. However, if it's
[`False`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:False), it returns the `return ()`, action, so an I/O action that doesn't
do anything. Here's how we could rewrite the previous piece of code with
which we demonstrated [`getChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getChar) by using [`when`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:when):

In [97]:
import Control.Monad

main = do
    c <- getChar
    when (c /= ' ') $ do
        putChar c
        main

In [98]:
withStdin "hello sir" main

hello

So as you can see, it's useful for encapsulating the
`if <something> then do <some I/O action> else return ()` pattern.

__[`sequence`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:sequence)__ takes a list of I/O actions and returns an I/O action that
will perform those actions one after the other. The result contained in
that I/O action will be a list of the results of all the I/O actions
that were performed. Its type signature is
`sequence :: [IO a] -> IO [a]`. Doing this:

In [99]:
main = do
    a <- getLine
    b <- getLine
    c <- getLine
    print [a,b,c]

Is exactly the same as doing this:.

In [100]:
main = do
    rs <- sequence [getLine, getLine, getLine]
    print rs

So `sequence [getLine, getLine, getLine]` makes an I/O action that will
perform [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) three times. If we bind that action to a name, the
result is a list of all the results, so in our case, a list of three
things that the user entered at the prompt.

A common pattern with [`sequence`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:sequence) is when we map functions like [`print`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:print) or
[`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) over lists. Doing `map print [1,2,3,4]` won't create an I/O
action. It will create a list of I/O actions, because that's like
writing `[print 1, print 2, print 3, print 4]`. If we want to transform
that list of I/O actions into an I/O action, we have to sequence it.

In [101]:
sequence (map print [1,2,3,4,5])

Line 1: Use mapM
Found:
sequence (map print [1, 2, 3, 4, 5])
Why not:
mapM print [1, 2, 3, 4, 5]

1
2
3
4
5
[(),(),(),(),()]

What's with the `[(),(),(),(),()]` at the end? Well, when we evaluate an
I/O action in GHCI, it's performed and then its result is printed out,
unless that result is `()`, in which case it's not printed out. That's why
evaluating `putStrLn "hehe"` in GHCI just prints out `hehe` (because the
contained result in `putStrLn "hehe"` is `()`). But when we do [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) in
GHCI, the result of that I/O action is printed out, because [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) has
a type of `IO String`.

Because mapping a function that returns an I/O action over a list and
then sequencing it is so common, the utility functions __[`mapM`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:mapM)__ and __`mapM_`__
were introduced. [`mapM`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:mapM) takes a function and a list, maps the function
over the list and then sequences it. `mapM_` does the same, only it
throws away the result later. We usually use `mapM_` when we don't care
what result our sequenced I/O actions have.

In [102]:
mapM print [1,2,3]

1
2
3
[(),(),()]

In [103]:
mapM_ print [1,2,3]

1
2
3

__[`forever`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forever)__ takes an I/O action and returns an I/O action that just repeats
the I/O action it got forever. It's located in `Control.Monad`. This
little program will indefinitely ask the user for some input and spit it
back to him, CAPSLOCKED:

In [104]:
import Control.Monad
import Data.Char

main = forever $ do
    putStr "Give me some input: "
    l <- getLine
    putStrLn $ map toUpper l

In [105]:
withStdin "capslock" $ catchPrint main

Give me some input: CAPSLOCK
Give me some input: <stdin>: hGetLine: end of file

__[`forM`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forM)__ (located in `Control.Monad`) is like [`mapM`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:mapM), only that it has its
parameters switched around. The first parameter is the list and the
second one is the function to map over that list, which is then
sequenced. Why is that useful? Well, with some creative use of lambdas
and *do* notation, we can do stuff like this:

In [106]:
import Control.Monad

main = do
    colors <- forM [1,2,3,4] (\a -> do
        putStrLn $ "Which color do you associate with the number " ++ show a ++ "?"
        color <- getLine
        return color)
    putStrLn "The colors that you associate with 1, 2, 3 and 4 are: "
    mapM_ putStrLn colors

Line 4: Redundant return
Found:
do putStrLn
     $ "Which color do you associate with the number " ++ show a ++ "?"
   color <- getLine
   return color
Why not:
do putStrLn
     $ "Which color do you associate with the number " ++ show a ++ "?"
   getLine

The `(\a -> do ... )` is a function that takes a number and returns an
I/O action. We have to surround it with parentheses, otherwise the
lambda thinks the last two I/O actions belong to it. Notice that we do
`return color` in the inside *do* block. We do that so that the I/O action
which the *do* block defines has the result of our color contained
within it. We actually didn't have to do that, because [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) already
has that contained within it. Doing `color <- getLine` and then
`return color` is just unpacking the result from [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) and then repackaging it
again, so it's the same as just doing [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine). The [`forM`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forM) (called with its
two parameters) produces an I/O action, whose result we bind to `colors`.
`colors` is just a normal list that holds strings. At the end, we print
out all those colors by doing `mapM putStrLn colors`.

You can think of [`forM`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forM) as meaning: make an I/O action for every element
in this list. What each I/O action will do can depend on the element
that was used to make the action. Finally, perform those actions and
bind their results to something. We don't have to bind it, we can also
just throw it away.

In [107]:
withStdin (unlines ["white", "blue", "red", "orange"]) main

Which color do you associate with the number 1?
Which color do you associate with the number 2?
Which color do you associate with the number 3?
Which color do you associate with the number 4?
The colors that you associate with 1, 2, 3 and 4 are: 
white
blue
red
orange

We could have actually done that without [`forM`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forM), only with [`forM`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forM) it's more
readable. Normally we write [`forM`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forM) when we want to map and sequence some
actions that we define there on the spot using *do* notation. In the
same vein, we could have replaced the last line with
`forM colors putStrLn`.

In this section, we learned the basics of input and output. We also
found out what I/O actions are, how they enable us to do input and
output and when they are actually performed. To reiterate, I/O actions
are values much like any other value in Haskell. We can pass them as
parameters to functions and functions can return I/O actions as results.
What's special about them is that if they fall into the `main` function
(or are the result in a GHCI line), they are performed. And that's when
they get to write stuff on your screen or play Yakety Sax through your
speakers. Each I/O action can also encapsulate a result with which it
tells you what it got from the real world.

Don't think of a function like [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) as a function that takes a
string and prints it to the screen. Think of it as a function that takes
a string and returns an I/O action. That I/O action will, when
performed, print beautiful poetry to your terminal.

Files and streams
-----------------

<img src="img/streams.png" title="streams" style="background-color:white;float:right;margin-left:2em;" />

[`getChar`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getChar) is an I/O action that reads a single character from the
terminal. [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) is an I/O action that reads a line from the terminal.
These two are pretty straightforward and most programming languages have
some functions or statements that are parallel to them. But now, let's
meet __[`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents)__. [`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents) is an I/O action that reads everything
from the standard input until it encounters an end-of-file character.
Its type is `getContents :: IO String`. What's cool about [`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents) is
that it does lazy I/O. When we do `foo <- getContents`, it doesn't read
all of the input at once, store it in memory and then bind it to `foo`.
No, it's lazy! It'll say: *"Yeah yeah, I'll read the input from the
terminal later as we go along, when you really need it!"*.

[`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents) is really useful when we're piping the output from one
program into the input of our program. In case you don't know how piping
works in unix-y systems, here's a quick primer. Let's make a text file
that contains the following little haiku:

In [108]:
writeFile "/tmp/haiku.txt" $ unlines 
  [ "I'm a lil' teapot"
  , "What's with that airplane food, huh?"
  , "It's so small, tasteless"
  ]

Yeah, the haiku sucks, what of it? If anyone knows of any good haiku
tutorials, let me know.

Now, recall the little program we wrote when we were introducing the
[`forever`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forever) function. It prompted the user for a line, returned it to him in
CAPSLOCK and then did that all over again, indefinitely. Just so you
don't have to scroll all the way back, here it is again:

In [109]:
import Control.Monad
import Data.Char

main = forever $ do
    putStr "Give me some input: "
    l <- getLine
    putStrLn $ map toUpper l

Check it out, booyaka!

In [110]:
readFile "/tmp/haiku.txt" >>= flip withStdin (catchPrint main)

Give me some input: I'M A LIL' TEAPOT
Give me some input: WHAT'S WITH THAT AIRPLANE FOOD, HUH?
Give me some input: IT'S SO SMALL, TASTELESS
Give me some input: <stdin>: hGetLine: end of file

So what we're essentially doing with that use of [`forever`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:forever) is taking the
input and transforming it into some output. That's why we can use
[`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents) to make our program even shorter and better:

In [111]:
import Data.Char

main = do
    contents <- getContents
    putStr (map toUpper contents)

We run the [`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents) I/O action and name the string it produces
`contents`. Then, we map [`toUpper`](https://hackage.haskell.org/package/base/docs/Data-Char.html#v:toUpper) over that string and print that to the
terminal. Keep in mind that because strings are basically lists, which
are lazy, and [`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents) is I/O lazy, it won't try to read the whole
content at once and store it into memory before printing out the
capslocked version. Rather, it will print out the capslocked version as
it reads it, because it will only read a line from the input when it
really needs to.

In [112]:
readFile "/tmp/haiku.txt" >>= flip withStdin main

I'M A LIL' TEAPOT
WHAT'S WITH THAT AIRPLANE FOOD, HUH?
IT'S SO SMALL, TASTELESS

Cool, it works.

In [113]:
withStdin (unlines ["hey ho", "lets go"]) main

HEY HO
LETS GO

As you can see, it
prints out our capslocked input back to us line by line. When the result
of [`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile) is bound to `contents`, it's not represented in memory as a
real string, but more like a promise that it will produce the string
eventually. When we map [`toUpper`](https://hackage.haskell.org/package/base/docs/Data-Char.html#v:toUpper) over `contents`, that's also a promise to
map that function over the eventual contents. And finally when [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr)
happens, it says to the previous promise: *"Hey, I need a capslocked
line!"*. It doesn't have any lines yet, so it says to `contents`: *"Hey,
how about actually getting a line from the terminal?"*. So that's when
[`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile) actually reads from the file and gives a line to the
code that asked it to produce something tangible. That code then maps
[`toUpper`](https://hackage.haskell.org/package/base/docs/Data-Char.html#v:toUpper) over that line and gives it to [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr), which prints it. And
then, [`putStr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStr) says: *"Hey, I need the next line, come on!"* and this
repeats until there's no more input, which is signified by an
end-of-file character.

Let's make program that takes some input and prints out only those lines
that are shorter than 10 characters. Observe:

In [114]:
main = do
    contents <- getContents
    putStr (shortLinesOnly contents)

shortLinesOnly :: String -> String
shortLinesOnly input =
    let allLines = lines input
        shortLines = filter (\line -> length line < 10) allLines
        result = unlines shortLines
    in  result

We've made our I/O part of the program as short as possible. Because our
program is supposed to take some input and print out some output based
on the input, we can implement it by reading the input contents, running
a function on them and then printing out what the function gave back.

The `shortLinesOnly` function works like this: it takes a string, like
`"short\nlooooooooooooooong\nshort again"`. That string has three lines,
two of them are short and the middle one is long. It runs the lines
function on that string, which converts it to
`["short", "looooooooooooooong", "short again"]`, which we then bind to the name
`allLines`. That list of string is then filtered so that only those lines
that are shorter than 10 characters remain in the list, producing
`["short", "short again"]`. And finally, [`unlines`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:unlines) joins that list into a
single newline delimited string, giving `"short\nshort again"`. Let's
give it a go.

In [115]:
writeFile "/tmp/shortlines.txt" $ unlines 
  [ "i'm short"
  , "so am i"
  , "i am a loooooooooong line!!!"
  , "yeah i'm long so what hahahaha!!!!!!"
  , "short line"
  , "loooooooooooooooooooooooooooong"
  , "short"
  ]

In [116]:
readFile "/tmp/shortlines.txt" >>= flip withStdin main

i'm short
so am i
short

As the output, we only get the short lines.

This pattern of getting some string from the input, transforming it with
a function and then outputting that is so common that there exists a
function which makes that even easier, called __[`interact`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:interact)__. [`interact`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:interact) takes a
function of type `String -> String` as a parameter and returns an I/O
action that will take some input, run that function on it and then print
out the function's result. Let's modify our program to use that.

In [117]:
main = interact shortLinesOnly

shortLinesOnly :: String -> String
shortLinesOnly input =
    let allLines = lines input
        shortLines = filter (\line -> length line < 10) allLines
        result = unlines shortLines
    in  result

Just to show that this can be achieved in much less code (even though it
will be less readable) and to demonstrate our function composition
skill, we're going to rework that a bit further.

In [118]:
main = interact $ unlines . filter ((<10) . length) . lines

In [119]:
readFile "/tmp/shortlines.txt" >>= flip withStdin main

i'm short
so am i
short

Wow, we actually reduced that to just one line, which is pretty cool!

[`interact`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:interact) can be used to make programs that are piped some contents into
them and then dump some result out or it can be used to make programs
that appear to take a line of input from the user, give back some result
based on that line and then take another line and so on. There isn't
actually a real distinction between the two, it just depends on how the
user is supposed to use them.

Let's make a program that continuously reads a line and then tells us if
the line is a palindrome or not. We could just use [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) to read a
line, tell the user if it's a palindrome and then run `main` all over
again. But it's simpler if we use [`interact`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:interact). When using [`interact`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:interact), think
about what you need to do to transform some input into the desired
output. In our case, we have to replace each line of the input with
either `"palindrome"` or `"not a palindrome"`. So we have to write a
function that transforms something like `"elephant\nABCBA\nwhatever"`
into `"not a palindrome\npalindrome\nnot a palindrome"`. Let's do this!

In [120]:
respondPalindromes contents = unlines (map (\xs -> if isPalindrome xs then "palindrome" else "not a palindrome") (lines contents))
    where   isPalindrome xs = xs == reverse xs

Let's write this in point-free.

In [121]:
respondPalindromes = unlines . map (\xs -> if isPalindrome xs then "palindrome" else "not a palindrome") . lines
    where   isPalindrome xs = xs == reverse xs

Pretty straightforward. First it turns something like
`"elephant\nABCBA\nwhatever"` into `["elephant", "ABCBA", "whatever"]` and
then it maps that lambda over it, giving
`["not a palindrome", "palindrome", "not a palindrome"]`
and then [`unlines`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:unlines) joins that list into
a single, newline delimited string. Now we can do

In [122]:
main = interact respondPalindromes

Let's test this out:

In [123]:
withStdin (unlines ["hehe", "ABCBA", "cookie"]) main

not a palindrome
palindrome
not a palindrome

Even though we made a program that transforms one big string of input into another, it acts like we made a program that does it line by line. That's because Haskell is lazy and it wants to print the first line of the result string, but it can't because it doesn't have the first line of the input yet. So as soon as we give it the first line of input, it prints the first line of the output. We get out of the program by issuing an end-of-line character.

We can also use this program by just piping a file into it. Let's say we
have this file:

In [124]:
writeFile "/tmp/words.txt" $ unlines
  [ "dogaroo"
  , "radar"
  , "rotor"
  , "madam"
  ]

and we save it as `/tmp/words.txt`. This is what we get by piping it into our
program:

In [125]:
readFile "/tmp/words.txt" >>= flip withStdin main

not a palindrome
palindrome
palindrome
palindrome

Again, we get the same output as if we had run our program and put in
the words ourselves at the standard input. We just don't see the input
that `palindromes.hs` because the input came from the file and not from us
typing the words in.

So now you probably see how lazy I/O works and how we can use it to our
advantage. You can just think in terms of what the output is supposed to
be for some given input and write a function to do that transformation.
In lazy I/O, nothing is eaten from the input until it absolutely has to
be because what we want to print right now depends on that input.

So far, we've worked with I/O by printing out stuff to the terminal and
reading from it. But what about reading and writing files? Well, in a
way, we've already been doing that. One way to think about reading from
the terminal is to imagine that it's like reading from a (somewhat
special) file. Same goes for writing to the terminal, it's kind of like
writing to a file. We can call these two files [`stdout`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:stdout) and [`stdin`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:stdin), meaning
*standard output* and *standard input*, respectively. Keeping that in
mind, we'll see that writing to and reading from files is very much like
writing to the standard output and reading from the standard input.

We'll start off with a really simple program that opens a file called
`girlfriend.txt`, which contains a verse from Avril Lavigne's \#1 hit
*Girlfriend*, and just prints out out to the terminal. Here's
`girlfriend.txt`:

In [126]:
writeFile "/tmp/girlfriend.txt" $ unlines
  [ "Hey! Hey! You! You!"
  , "I don't like your girlfriend!"
  , "No way! No way!"
  , "I think you need a new one!"
  ]

And here's our program:

In [127]:
import System.IO

main = do
    handle <- openFile "/tmp/girlfriend.txt" ReadMode
    contents <- hGetContents handle
    putStr contents
    hClose handle

Running it, we get the expected result:

In [128]:
main

Hey! Hey! You! You!
I don't like your girlfriend!
No way! No way!
I think you need a new one!

Let's go over this line by line. The first line is just four
exclamations, to get our attention. In the second line, Avril tells us
that she doesn't like our current romantic partner. The third line
serves to emphasize that disapproval, whereas the fourth line suggests
we should seek out a new girlfriend.

Let's also go over the program line by line! Our program is several I/O
actions glued together with a *do* block. In the first line of the *do*
block, we notice a new function called __[`openFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:openFile)__. This is its type
signature: `openFile :: FilePath -> IOMode -> IO Handle`. If you read
that out loud, it states: [`openFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:openFile) takes a file path and an [`IOMode`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:IOMode) and
returns an I/O action that will open a file and have the file's
associated handle encapsulated as its result.

[`FilePath`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:FilePath) is just a [type synonym](http://learnyouahaskell.com/making-our-own-types-and-typeclasses#type-synonyms) for [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String),
simply defined as:

<pre><code>type FilePath = String</code></pre>

[`IOMode`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:IOMode) is a type that's defined like this:

<pre><code>data IOMode = ReadMode | WriteMode | AppendMode | ReadWriteMode</code></pre>

<img src="img/file.png" title="A FILE IN A CAKE!!!" style="background-color:white;float:left;margin-right:2em;" />

Just like our type that represents the seven possible values for the
days of the week, this type is an enumeration that represents what we
want to do with our opened file. Very simple. Just note that this type
is [`IOMode`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:IOMode) and not `IO Mode`. `IO Mode` would be the type of an I/O action
that has a value of some type `Mode` as its result, but [`IOMode`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:IOMode) is just a
simple enumeration.

Finally, it returns an I/O action that will open the specified file in
the specified mode. If we bind that action to something we get a [`Handle`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:Handle).
A value of type [`Handle`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:Handle) represents where our file is. We'll use that
handle so we know which file to read from. It would be stupid to read a
file but not bind that read to a handle because we wouldn't be able to
do anything with the file. So in our case, we bound the handle to
[`handle`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:handle).

In the next line, we see a function called __[`hGetContents`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetContents)__. It takes a
[`Handle`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:Handle), so it knows which file to get the contents from and returns an
`IO String` — an I/O action that holds as its result the contents of the
file. This function is pretty much like [`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents). The only difference
is that [`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents) will automatically read from the standard input
(that is from the terminal), whereas [`hGetContents`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetContents) takes a file handle
which tells it which file to read from. In all other respects, they work
the same. And just like [`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents), [`hGetContents`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetContents) won't attempt to read
the file at once and store it in memory, but it will read it as needed.
That's really cool because we can treat `contents` as the whole contents
of the file, but it's not really loaded in memory. So if this were a
really huge file, doing [`hGetContents`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetContents) wouldn't choke up our memory, but
it would read only what it needed to from the file, when it needed to.

Note the difference between the handle used to identify a file and the
contents of the file, bound in our program to [`handle`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:handle) and `contents`. The
handle is just something by which we know what our file is. If you
imagine your whole file system to be a really big book and each file is
a chapter in the book, the handle is a bookmark that shows where you're
currently reading (or writing) a chapter, whereas the contents are the
actual chapter.

With `putStr contents` we just print the contents out to the standard
output and then we do __[`hClose`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hClose)__, which takes a handle and returns an I/O
action that closes the file. You have to close the file yourself after
opening it with openFile!

Another way of doing what we just did is to use the __[`withFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:withFile)__ function,
which has a type signature of
`withFile :: FilePath -> IOMode -> (Handle -> IO a) -> IO a`.
It takes a path to a file, an [`IOMode`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:IOMode) and
then it takes a function that takes a handle and returns some I/O
action. What it returns is an I/O action that will open that file, do
something we want with the file and then close it. The result
encapsulated in the final I/O action that's returned is the same as the
result of the I/O action that the function we give it returns. This
might sound a bit complicated, but it's really simple, especially with
lambdas, here's our previous example rewritten to use [`withFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:withFile):

In [129]:
import System.IO

main =
    withFile "/tmp/girlfriend.txt" ReadMode (\handle -> do
        contents <- hGetContents handle
        putStr contents)

As you can see, it's very similar to the previous piece of code.
`(\handle -> ... )` is the function that takes a handle and returns an
I/O action and it's usually done like this, with a lambda. The reason it
has to take a function that returns an I/O action instead of just taking
an I/O action to do and then close the file is because the I/O action
that we'd pass to it wouldn't know on which file to operate. This way,
[`withFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:withFile) opens the file and then passes the handle to the function we
gave it. It gets an I/O action back from that function and then makes an
I/O action that's just like it, only it closes the file afterwards.
Here's how we can make our own [`withFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:withFile) function:

In [130]:
withFile' :: FilePath -> IOMode -> (Handle -> IO a) -> IO a
withFile' path mode f = do
    handle <- openFile path mode
    result <- f handle
    hClose handle
    return result

<img src="img/edd.png" title="butter toast" style="background-color:white;float:right;margin-left:2em;" />

We know the result will be an I/O action so we can just start off with a
*do*. First we open the file and get a [`handle`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:handle) from it. Then, we apply
handle to our function to get back the I/O action that does all the
work. We bind that action to `result`, close the handle and then do
`return result`. By [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return)ing the result encapsulated in the I/O action that we
got from `f`, we make it so that our I/O action encapsulates the same
result as the one we got from `f handle`. So if `f handle` returns an action
that will read a number of lines from the standard input and write them
to a file and have as its result encapsulated the number of lines it
read, if we used that with `withFile'`, the resulting I/O action would
also have as its result the number of lines read.

Just like we have [`hGetContents`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetContents) that works like [`getContents`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getContents) but for a
specific file, there's also __[`hGetLine`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetLine)__, __[`hPutStr`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hPutStr)__, __[`hPutStrLn`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hPutStrLn)__, __[`hGetChar`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetChar)__, etc.
They work just like their counterparts without the *h*, only they take a
handle as a parameter and operate on that specific file instead of
operating on standard input or standard output. Example: [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn) is a
function that takes a string and returns an I/O action that will print
out that string to the terminal and a newline after it. [`hPutStrLn`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hPutStrLn) takes
a handle and a string and returns an I/O action that will write that
string to the file associated with the handle and then put a newline
after it. In the same vein, [`hGetLine`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetLine) takes a handle and returns an I/O
action that reads a line from its file.

Loading files and then treating their contents as strings is so common
that we have these three nice little functions to make our work even
easier:

__[`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile)__ has a type signature of `readFile :: FilePath -> IO String`.
Remember, [`FilePath`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:FilePath) is just a fancy name for [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String). [`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile) takes a
path to a file and returns an I/O action that will read that file
(lazily, of course) and bind its contents to something as a string. It's
usually more handy than doing [`openFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:openFile) and binding it to a handle and
then doing [`hGetContents`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hGetContents). Here's how we could have written our previous
example with [`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile):

In [131]:
import System.IO

main = do
    contents <- readFile "/tmp/girlfriend.txt"
    putStr contents

Because we don't get a handle with which to identify our file, we can't
close it manually, so Haskell does that for us when we use [`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile).

__[`writeFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:writeFile)__ has a type of `writeFile :: FilePath -> String -> IO ()`. It
takes a path to a file and a string to write to that file and returns an
I/O action that will do the writing. If such a file already exists, it
will be stomped down to zero length before being written on. Here's how
to turn `girlfriend.txt` into a CAPSLOCKED version and write it to
`girlfriendcaps.txt`:

In [132]:
import System.IO
import Data.Char

main = do
    contents <- readFile "/tmp/girlfriend.txt"
    writeFile "/tmp/girlfriendcaps.txt" (map toUpper contents)

In [133]:
main

In [134]:
putStrLn =<< readFile "/tmp/girlfriendcaps.txt"

HEY! HEY! YOU! YOU!
I DON'T LIKE YOUR GIRLFRIEND!
NO WAY! NO WAY!
I THINK YOU NEED A NEW ONE!

__[`appendFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:appendFile)__ has a type signature that's just like [`writeFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:writeFile), only
[`appendFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:appendFile) doesn't truncate the file to zero length if it already exists
but it appends stuff to it.

Let's say we have a file `todo.txt` that has one task per line that we
have to do. Now let's make a program that takes a line from the standard
input and adds that to our to-do list.

In [135]:
import System.IO

main = do     
    todoItem <- getLine  
    appendFile "/tmp/todo.txt" (todoItem ++ "\n")

In [136]:
writeFile "/tmp/todo.txt" ""
withStdin "Iron the dishes" main
withStdin "Dust the dog" main
withStdin "Take salad out of the oven" main

In [137]:
putStrLn =<< readFile "/tmp/todo.txt"

Iron the dishes
Dust the dog
Take salad out of the oven

We needed to add the `"\n"` to the end of each line because [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine)
doesn't give us a newline character at the end.

Ooh, one more thing. We talked about how doing
`contents <- hGetContents handle`
doesn't cause the whole file to be read at once and stored
in-memory. It's I/O lazy, so doing this:

In [138]:
main = do
    withFile "/tmp/something.txt" ReadMode (\handle -> do
        contents <- hGetContents handle
        putStr contents)

is actually like connecting a pipe from the file to the output. Just
like you can think of lists as streams, you can also think of files as
streams. This will read one line at a time and print it out to the
terminal as it goes along. So you may be asking, how wide is this pipe
then? How often will the disk be accessed? Well, for text files, the
default buffering is line-buffering usually. That means that the
smallest part of the file to be read at once is one line. That's why in
this case it actually reads a line, prints it to the output, reads the
next line, prints it, etc. For binary files, the default buffering is
usually block-buffering. That means that it will read the file chunk by
chunk. The chunk size is some size that your operating system thinks is
cool.

You can control how exactly buffering is done by using the [`hSetBuffering`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hSetBuffering)
function. It takes a handle and a [`BufferMode`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:BufferMode) and returns an I/O action
that sets the buffering. [`BufferMode`](https://hackage.haskell.org/package/base/docs/System-IO.html#t:BufferMode) is a simple enumeration data type
and the possible values it can hold are: [`NoBuffering`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:NoBuffering), [`LineBuffering`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:LineBuffering) or
`BlockBuffering (Maybe Int)`. The `Maybe Int` is for how big the chunk
should be, in bytes. If it's [`Nothing`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:Nothing), then the operating system
determines the chunk size. [`NoBuffering`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:NoBuffering) means that it will be read one
character at a time. [`NoBuffering`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:NoBuffering) usually sucks as a buffering mode
because it has to access the disk so much.

Here's our previous piece of code, only it doesn't read it line by line
but reads the whole file in chunks of 2048 bytes.

In [139]:
main = do
    withFile "/tmp/something.txt" ReadMode (\handle -> do
        hSetBuffering handle $ BlockBuffering (Just 2048)
        contents <- hGetContents handle
        putStr contents)

Reading files in bigger chunks can help if we want to minimize disk
access or when our file is actually a slow network resource.

We can also use __[`hFlush`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hFlush)__, which is a function that takes a handle and
returns an I/O action that will flush the buffer of the file associated
with the handle. When we're doing line-buffering, the buffer is flushed
after every line. When we're doing block-buffering, it's after we've
read a chunk. It's also flushed after closing a handle. That means that
when we've reached a newline character, the reading (or writing)
mechanism reports all the data so far. But we can use [`hFlush`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hFlush) to force
that reporting of data that has been read so far. After flushing, the
data is available to other programs that are running at the same time.

Think of reading a block-buffered file like this: your toilet bowl is
set to flush itself after it has one gallon of water inside it. So you
start pouring in water and once the gallon mark is reached, that water
is automatically flushed and the data in the water that you've poured in
so far is read. But you can flush the toilet manually too by pressing
the button on the toilet. This makes the toilet flush and all the water
(data) inside the toilet is read. In case you haven't noticed, flushing
the toilet manually is a metaphor for [`hFlush`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:hFlush). This is not a very great
analogy by programming analogy standards, but I wanted a real world
object that can be flushed for the punchline.

We already made a program to add a new item to our to-do list in
`todo.txt`, now let's make a program to remove an item. I'll just paste
the code and then we'll go over the program together so you see that
it's really easy. We'll be using a few new functions from
`System.Directory` and one new function from `System.IO`, but they'll all be
explained.

Anyway, here's the program for removing an item from `todo.txt`:

In [216]:
import System.IO
import System.Directory
import Data.List

main = do
    handle <- openFile "/tmp/todo.txt" ReadMode
    (tempName, tempHandle) <- openTempFile "/tmp" "todotemp"
    contents <- hGetContents handle
    let todoTasks = lines contents
        numberedTasks = zipWith (\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStrLn "These are your TO-DO items:"
    putStr $ unlines numberedTasks
    putStrLn "Which one do you want to delete?"
    numberString <- getLine
    let number = read numberString
        newTodoItems = delete (todoTasks !! number) todoTasks
    hPutStr tempHandle $ unlines newTodoItems
    hClose handle
    hClose tempHandle
    removeFile "/tmp/todo.txt"
    renameFile tempName "/tmp/todo.txt"

At first, we just open `todo.txt` in read mode and bind its handle to
[`handle`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:handle).

Next up, we use a function that we haven't met before which is from
`System.IO` — __[`openTempFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:openTempFile)__. Its name is pretty self-explanatory. It takes
a path to a temporary directory and a template name for a file and opens
a temporary file. We used `"."` for the temporary directory, because .
denotes the current directory on just about any OS. We used `"temp"` as
the template name for the temporary file, which means that the temporary
file will be named *temp* plus some random characters. It returns an I/O
action that makes the temporary file and the result in that I/O action
is a pair of values: the name of the temporary file and a handle. We
could just open a normal file called `todo2.txt` or something like that
but it's better practice to use [`openTempFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:openTempFile) so you know you're probably
not overwriting anything.

The reason we didn't use `getCurrentDirectory` to get the current
directory and then pass it to [`openTempFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:openTempFile) but instead just passed `"."`
to [`openTempFile`](https://hackage.haskell.org/package/base/docs/System-IO.html#v:openTempFile) is because [`.`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:.) refers to the current directory on
unix-like system and Windows

Next up, we bind the contents of `todo.txt` to `contents`. Then, split
that string into a list of strings, each string one line. So `todoTasks`
is now something like
`["Iron the dishes", "Dust the dog", "Take salad out of the oven"]`.
We zip the numbers from 0 onwards and that list with
a function that takes a number, like 3, and a string, like `"hey"` and
returns `"3 - hey"`, so `numberedTasks` is
`["0 - Iron the dishes", "1 - Dust the dog" ...`.
We join that list of strings into a single newline
delimited string with [`unlines`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:unlines) and print that string out to the terminal.
Note that instead of doing that, we could have also done
`mapM putStrLn numberedTasks`.

We ask the user which one they want to delete and wait for them to enter
a number. Let's say they want to delete number 1, which is `Dust the dog`,
so they punch in `1`. `numberString` is now `"1"` and because we want a
number, not a string, we run [`read`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:read) on that to get `1` and bind that to
`number`.

Remember the [`delete`](https://hackage.haskell.org/package/base/docs/Data-List.html#v:delete) and `!!` functions from `Data.List`. `!!` returns an
element from a list with some index and [`delete`](https://hackage.haskell.org/package/base/docs/Data-List.html#v:delete) deletes the first
occurence of an element in a list and returns a new list without that
occurence. `(todoTasks !! number)` (number is now `1`) returns
`"Dust the dog"`. We bind `todoTasks` without the first occurence of `"Dust the dog"` to
`newTodoItems` and then join that into a single string with [`unlines`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:unlines) before
writing it to the temporary file that we opened. The old file is now
unchanged and the temporary file contains all the lines that the old one
does, except the one we deleted.

After that we close both the original and the temporary files and then
we remove the original one with __`removeFile`__, which, as you can see, takes
a path to a file and deletes it. After deleting the old `todo.txt`, we
use __`renameFile`__ to rename the temporary file to `todo.txt`. Be careful,
`removeFile` and `renameFile` (which are both in `System.Directory` by the
way) take file paths as their parameters, not handles.

And that's that! We could have done this in even fewer lines, but we
were very careful not to overwrite any existing files and politely asked
the operating system to tell us where we can put our temporary file.
Let's give this a go!

In [217]:
writeFile "/tmp/todo.txt" $ unlines
  [ "Iron the dishes"
  , "Dust the dog"
  , "Take salad out of the oven"
  ]

In [218]:
withStdin "1" main

These are your TO-DO items:
0 - Iron the dishes
1 - Dust the dog
2 - Take salad out of the oven
Which one do you want to delete?

In [219]:
putStrLn =<< readFile "/tmp/todo.txt"

Iron the dishes
Take salad out of the oven

In [220]:
withStdin "0" main

These are your TO-DO items:
0 - Iron the dishes
1 - Take salad out of the oven
Which one do you want to delete?

In [221]:
putStrLn =<< readFile "/tmp/todo.txt"

Take salad out of the oven

Command line arguments
----------------------

<img src="img/arguments.png" title="COMMAND LINE ARGUMENTS!!! ARGH" style="background-color:white;float:right;margin-left:2em;"/>

Dealing with command line arguments is pretty much a necessity if you
want to make a script or application that runs on a terminal. Luckily,
Haskell's standard library has a nice way of getting command line
arguments of a program.

In the previous section, we made one program for adding a to-do item to
our to-do list and one program for removing an item. There are two
problems with the approach we took. The first one is that we just
hardcoded the name of our to-do file in our code. We just decided that
the file will be named `todo.txt` and that the user will never have a
need for managing several to-do lists.

One way to solve that is to always ask the user which file they want to
use as their to-do list. We used that approach when we wanted to know
which item the user wants to delete. It works, but it's not so good,
because it requires the user to run the program, wait for the program to
ask something and then tell that to the program. That's called an
interactive program and the difficult bit with interactive command line
programs is this — what if you want to automate the execution of that
program, like with a batch script? It's harder to make a batch script
that interacts with a program than a batch script that just calls one
program or several of them.

That's why it's sometimes better to have the user tell the program what
they want when they run the program, instead of having the program ask
the user once it's run. And what better way to have the user tell the
program what they want it to do when they run it than via command line
arguments!

The `System.Environment` module has two cool I/O actions. One is __[`getArgs`](https://hackage.haskell.org/package/base/docs/System-Environment.html#v:getArgs)__,
which has a type of `getArgs :: IO [String]` and is an I/O action that
will get the arguments that the program was run with and have as its
contained result a list with the arguments. __[`getProgName`](https://hackage.haskell.org/package/base/docs/System-Environment.html#v:getProgName)__ has a type of
`getProgName :: IO String` and is an I/O action that contains the program
name.

Here's a small program that demonstrates how these two work:

In [222]:
 import System.Environment
 import Data.List

 main = do
    args <- getArgs
    progName <- getProgName
    putStrLn "The arguments are:"
    mapM_ putStrLn args
    putStrLn "The program name is:"
    putStrLn progName

We bind [`getArgs`](https://hackage.haskell.org/package/base/docs/System-Environment.html#v:getArgs) and `progName` to `args` and `progName`. We say
`The arguments are:` and then for every argument in `args`, we do [`putStrLn`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:putStrLn). Finally, we
also print out the program name. Let's compile this as `arg-test`.

In [223]:
withArgs ["first", "second", "w00t", "multi word arg"] main

The arguments are:
first
second
w00t
multi word arg
The program name is:
ihaskell

Nice. Armed with this knowledge you could create some cool command line
apps. In fact, let's go ahead and make one. In the previous section, we
made a separate program for adding tasks and a separate program for
deleting them. Now, we're going to join that into one program, what it
does will depend on the command line arguments. We're also going to make
it so it can operate on different files, not just `todo.txt`.

We'll call it simply `todo` and it'll be able to do (haha!) three
different things:

-   View tasks
-   Add tasks
-   Delete tasks

We're not going to concern ourselves with possible bad input too much
right now.

Our program will be made so that if we want to add the task
`Find the magic sword of power` to the file `todo.txt`, we have to punch in
`todo add todo.txt "Find the magic sword of power"` in our terminal. To view
the tasks we'll just do `todo view todo.txt` and to remove the task with
the index of 2, we'll do `todo remove todo.txt 2`.

We'll start by making a dispatch association list. It's going to be a
simple association list that has command line arguments as keys and
functions as their corresponding values. All these functions will be of
type `[String] -> IO ()`. They're going to take the argument list as a
parameter and return an I/O action that does the viewing, adding,
deleting, etc.

<pre><code>import System.Environment
import System.Directory
import System.IO
import Data.List

dispatch :: [(String, [String] -> IO ())]
dispatch =  [ ("add", add)
            , ("view", view)
            , ("remove", remove)
            ]</code></pre>

We have yet to define `main`, `add`, `view` and `remove`, so let's start with
`main`:

<pre><code>main = do
    (command:args) <- getArgs
    let (Just action) = lookup command dispatch
    action args</code></pre>

First, we get the arguments and bind them to `(command:args)`. If you
remember your pattern matching, this means that the first argument will
get bound to `command` and the rest of them will get bound to `args`. If we
call our program like `todo add todo.txt "Spank the monkey"`, `command` will
be `"add"` and args will be `["todo.txt", "Spank the monkey"]`.

In the next line, we look up our command in the dispatch list. Because
`"add"` points to `add`, we get `Just add` as a result. We use pattern
matching again to extract our function out of the [`Maybe`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Maybe). What happens if
our command isn't in the dispatch list? Well then the lookup will return
[`Nothing`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:Nothing), but we said we won't concern ourselves with failing gracefully
too much, so the pattern matching will fail and our program will throw a
fit.

Finally, we call our `action` function with the rest of the argument list.
That will return an I/O action that either adds an item, displays a list
of items or deletes an item and because that action is part of the `main`
*do* block, it will get performed. If we follow our concrete example so
far and our `action` function is `add`, it will get called with `args` (so
`["todo.txt", "Spank the monkey"]`) and return an I/O action that adds
`Spank the monkey` to `todo.txt`.

Great! All that's left now is to implement `add`, `view` and `remove`. Let's
start with `add`:

<code><pre>add :: [String] -> IO ()
add [fileName, todoItem] = appendFile fileName (todoItem ++ "\n")</code></pre>

If we call our program like `todo add todo.txt "Spank the monkey"`, the
`"add"` will get bound to `command` in the first pattern match in the `main`
block, whereas `["todo.txt", "Spank the monkey"]` will get passed to the
function that we get from the dispatch list. So, because we're not
dealing with bad input right now, we just pattern match against a list
with those two elements right away and return an I/O action that appends
that line to the end of the file, along with a newline character.

Next, let's implement the list viewing functionality. If we want to view
the items in a file, we do `todo view todo.txt`. So in the first pattern
match, `command` will be `"view"` and `args` will be `["todo.txt"]`.

<pre><code>view :: [String] -> IO ()
view [fileName] = do
    contents <- readFile fileName
    let todoTasks = lines contents
        numberedTasks = zipWith (\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStr $ unlines numberedTasks</code></pre>

We already did pretty much the same thing in the program that only
deleted tasks when we were displaying the tasks so that the user can
choose one for deletion, only here we just display the tasks.

And finally, we're going to implement `remove`. It's going to be very
similar to the program that only deleted the tasks, so if you don't
understand how deleting an item here works, check out the explanation
under that program. The main difference is that we're not hardcoding
`todo.txt` but getting it as an argument. We're also not prompting the
user for the task number to delete, we're getting it as an argument.

<pre><code>remove :: [String] -> IO ()
remove [fileName, numberString] = do
    handle <- openFile fileName ReadMode
    (tempName, tempHandle) <- openTempFile "." "temp"
    contents <- hGetContents handle
    let number = read numberString
        todoTasks = lines contents
        newTodoItems = delete (todoTasks !! number) todoTasks
    hPutStr tempHandle $ unlines newTodoItems
    hClose handle
    hClose tempHandle
    removeFile fileName
    renameFile tempName fileName</code></pre>

We opened up the file based on `fileName` and opened a temporary file,
deleted the line with the index that the user wants to delete, wrote
that to the temporary file, removed the original file and renamed the
temporary file back to `fileName`.

Here's the whole program at once, in all its glory!

In [225]:
import System.Environment
import System.Directory
import System.IO
import Data.List

dispatch :: [(String, [String] -> IO ())]
dispatch =  [ ("add", add)
            , ("view", view)
            , ("remove", remove)
            ]

main = do
    (command:args) <- getArgs
    let (Just action) = lookup command dispatch
    action args

add :: [String] -> IO ()
add [fileName, todoItem] = appendFile fileName (todoItem ++ "\n")

view :: [String] -> IO ()
view [fileName] = do
    contents <- readFile fileName
    let todoTasks = lines contents
        numberedTasks = zipWith (\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStr $ unlines numberedTasks

remove :: [String] -> IO ()
remove [fileName, numberString] = do
    handle <- openFile fileName ReadMode
    (tempName, tempHandle) <- openTempFile "/tmp" "todotemp"
    contents <- hGetContents handle
    let number = read numberString
        todoTasks = lines contents
        newTodoItems = delete (todoTasks !! number) todoTasks
    hPutStr tempHandle $ unlines newTodoItems
    hClose handle
    hClose tempHandle
    removeFile fileName
    renameFile tempName fileName

<img src="img/salad.png" title="fresh baked salad" style="background-color:white;float:left;margin-right:2em;" />

To summarize our solution: we made a dispatch association that maps from
commands to functions that take some command line arguments and return
an I/O action. We see what the command is and based on that we get the
appropriate function from the dispatch list. We call that function with
the rest of the command line arguments to get back an I/O action that
will do the appropriate thing and then just perform that action!

In other languages, we might have implemented this with a big switch
case statement or whatever, but using higher order functions allows us
to just tell the dispatch list to give us the appropriate function and
then tell that function to give us an I/O action for some command line
arguments.

Let's try our app out!

In [226]:
writeFile "/tmp/todo.txt" $ unlines
  [ "Iron the dishes"
  , "Dust the dog"
  , "Take salad out of the oven"
  ]

In [227]:
withArgs ["view", "/tmp/todo.txt"] main

0 - Iron the dishes
1 - Dust the dog
2 - Take salad out of the oven

In [228]:
withArgs ["add", "/tmp/todo.txt", "Pick up children from drycleaners"] main

In [229]:
withArgs ["view", "/tmp/todo.txt"] main

0 - Iron the dishes
1 - Dust the dog
2 - Take salad out of the oven
3 - Pick up children from drycleaners

In [230]:
withArgs ["remove", "/tmp/todo.txt", "2"] main

In [231]:
withArgs ["view", "/tmp/todo.txt"] main

0 - Iron the dishes
1 - Dust the dog
2 - Pick up children from drycleaners

Another cool thing about this is that it's easy to add extra
functionality. Just add an entry in the dispatch association list and
implement the corresponding function and you're laughing! As an
exercise, you can try implementing a `bump` function that will take a file
and a task number and return an I/O action that bumps that task to the
top of the to-do list.

You could make this program fail a bit more gracefully in case of bad
input (for example, if someone runs `todo UP YOURS HAHAHAHA`) by making an
I/O action that just reports there has been an error (say,
`errorExit :: IO ()`) and then check for possible erroneous input and if there is
erroneous input, perform the error reporting I/O action. Another way is
to use exceptions, which we will meet soon.

<a name="randomness"></a>

Randomness
----------

<img src="img/random.png" title="this picture is the ultimate source of randomness and wackiness" style="background-color:white;float:right;margin-left:2em;"/>

Many times while programming, you need to get some random data. Maybe
you're making a game where a die needs to be thrown or you need to
generate some test data to test out your program. There are a lot of
uses for random data when programming. Well, actually, pseudo-random,
because we all know that the only true source of randomness is a monkey
on a unicycle with a cheese in one hand and its butt in the other. In
this section, we'll take a look at how to make Haskell generate
seemingly random data.

In most other programming languages, you have functions that give you
back some random number. Each time you call that function, you get back
a (hopefully) different random number. How about Haskell? Well,
remember, Haskell is a pure functional language. What that means is that
it has referential transparency. What THAT means is that a function, if
given the same parameters twice, must produce the same result twice.
That's really cool because it allows us to reason differently about
programs and it enables us to defer evaluation until we really need it.
If I call a function, I can be sure that it won't do any funny stuff
before giving me the results. All that matters are its results. However,
this makes it a bit tricky for getting random numbers. If I have a
function like this:

<pre><code>randomNumber :: (Num a) => a
randomNumber = 4</code></pre>

It's not very useful as a random number function because it will always
return `4`, even though I can assure you that the 4 is completely random,
because I used a die to determine it.

How do other languages make seemingly random numbers? Well, they take
various info from your computer, like the current time, how much and
where you moved your mouse and what kind of noises you made behind your
computer and based on that, give a number that looks really random. The
combination of those factors (that randomness) is probably different in
any given moment in time, so you get a different random number.

Ah. So in Haskell, we can make a random number then if we make a
function that takes as its parameter that randomness and based on that
returns some number (or other data type).

Enter the `System.Random` module. It has all the functions that satisfy
our need for randomness. Let's just dive into one of the functions it
exports then, namely __`random`__. Here's its type:
`random :: (RandomGen g, Random a) => g -> (a, g)`. Whoa! Some new typeclasses in this type
declaration up in here! The __`RandomGen`__ typeclass is for types that can
act as sources of randomness. The __`Random`__ typeclass is for things that
can take on random values. A boolean value can take on a random value,
namely [`True`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:True) or [`False`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:False). A number can also take up a plethora of different
random values. Can a function take on a random value? I don't think so,
probably not! If we try to translate the type declaration of `random` to
English, we get something like: it takes a random generator (that's our
source of randomness) and returns a random value and a new random
generator. Why does it also return a new generator as well as a random
value? Well, we'll see in a moment.

To use our `random` function, we have to get our hands on one of those
random generators. The `System.Random` module exports a cool type, namely
__`StdGen`__ that is an instance of the `RandomGen` typeclass. We can either
make a `StdGen` manually or we can tell the system to give us one based on
a multitude of sort of random stuff.

To manually make a random generator, use the __`mkStdGen`__ function. It has a
type of `mkStdGen :: Int -> StdGen`. It takes an integer and based on
that, gives us a random generator. Okay then, let's try using `random` and
`mkStdGen` in tandem to get a (hardly random) number.

In [157]:
import System.Random

random (mkStdGen 100)

(9216477508314497915,StdGen {unStdGen = SMGen 712633246999323047 2532601429470541125})

<pre><code>\<interactive\>:1:0:
    Ambiguous type variable `a' in the constraint:
      `Random a' arising from a use of `random' at \<interactive\>:1:0-20
    Probable fix: add a type signature that fixes these type variable(s)</code></pre>

What's this? Ah, right, the `random` function can return a value of any
type that's part of the `Random` typeclass, so we have to inform Haskell
what kind of type we want. Also let's not forget that it returns a
random value and a random generator in a pair.

In [158]:
random (mkStdGen 100) :: (Int, StdGen)

(9216477508314497915,StdGen {unStdGen = SMGen 712633246999323047 2532601429470541125})

Finally! A number that looks kind of random! The first component of the
tuple is our number whereas the second component is a textual
representation of our new random generator. What happens if we call
`random` with the same random generator again?

In [159]:
random (mkStdGen 100) :: (Int, StdGen)

(9216477508314497915,StdGen {unStdGen = SMGen 712633246999323047 2532601429470541125})

Of course. The same result for the same parameters. So let's try giving
it a different random generator as a parameter.

In [160]:
random (mkStdGen 949494) :: (Int, StdGen)

(-3721561930999131179,StdGen {unStdGen = SMGen 3529285061597105018 656261746462218851})

Alright, cool, great, a different number. We can use the type annotation
to get different types back from that function.

In [161]:
random (mkStdGen 949488) :: (Float, StdGen)

(0.28144592,StdGen {unStdGen = SMGen 2179256218976226416 4388660484137712901})

In [162]:
random (mkStdGen 949488) :: (Bool, StdGen)

(True,StdGen {unStdGen = SMGen 2179256218976226416 4388660484137712901})

In [163]:
random (mkStdGen 949488) :: (Integer, StdGen)

(-1991610726167140167,StdGen {unStdGen = SMGen 2179256218976226416 4388660484137712901})

Let's make a function that simulates tossing a coin three times. If
`random` didn't return a new generator along with a random value, we'd
have to make this function take three random generators as a parameter
and then return coin tosses for each of them. But that sounds wrong
because if one generator can make a random value of type [`Int`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Int) (which can
take on a load of different values), it should be able to make three
coin tosses (which can take on precisely eight combinations). So this is
where `random` returning a new generator along with a value really comes
in handy.

We'll represent a coin with a simple [`Bool`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Bool). [`True`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:True) is tails, [`False`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:False) is
heads.

In [164]:
threeCoins :: StdGen -> (Bool, Bool, Bool)
threeCoins gen =
    let (firstCoin, newGen) = random gen :: (Bool, StdGen)
        (secondCoin, newGen') = random newGen :: (Bool, StdGen)
        (thirdCoin, newGen'') = random newGen' :: (Bool, StdGen)
    in  (firstCoin, secondCoin, thirdCoin)

We call `random` with the generator we got as a parameter to get a coin
and a new generator. Then we call it again, only this time with our new
generator, to get the second coin. We do the same for the third coin.
Had we called it with the same generator every time, all the coins would
have had the same value and we'd only be able to get
`(False, False, False)` or `(True, True, True)` as a result.

In [165]:
threeCoins (mkStdGen 21)

(True,False,False)

In [166]:
threeCoins (mkStdGen 22)

(False,True,True)

In [167]:
threeCoins (mkStdGen 943)

(True,False,True)

In [168]:
threeCoins (mkStdGen 944)

(False,False,False)

Notice that we didn't have to do `random gen :: (Bool, StdGen)`. That's
because we already specified that we want booleans in the type
declaration of the function. That's why Haskell can infer that we want a
boolean value in this case.

So what if we want to flip four coins? Or five? Well, there's a function
called __`randoms`__ that takes a generator and returns an infinite sequence
of values based on that generator.

In [169]:
take 5 $ randoms (mkStdGen 11) :: [Int]

[-8691282579076269733,-6401397088113744335,-7708536135073118066,1428399377092000640,-6279174627358091186]

In [170]:
take 5 $ randoms (mkStdGen 11) :: [Bool]

[True,True,False,False,False]

In [171]:
take 5 $ randoms (mkStdGen 11) :: [Float]

[6.3991964e-2,0.3095299,0.95818657,0.636883,0.10283136]

Why doesn't `randoms` return a new generator as well as a list? We could
implement the `randoms` function very easily like this:

In [172]:
:extension ExplicitForAll
:extension ScopedTypeVariables

randoms' :: forall g a. (RandomGen g, Random a) => g -> [a]
randoms' gen = let (value :: a, newGen :: g) = random gen in value:randoms' newGen

A recursive definition. We get a random value and a new generator from
the current generator and then make a list that has the value as its
head and random numbers based on the new generator as its tail. Because
we have to be able to potentially generate an infinite amount of
numbers, we can't give the new random generator back.

We could make a function that generates a finite stream of numbers and a
new generator like this:

In [173]:
:extension ExplicitForAll
:extension ScopedTypeVariables

finiteRandoms :: forall g a n. (RandomGen g, Random a, Num n, Eq n) => n -> g -> ([a], g)
finiteRandoms 0 gen = ([], gen)
finiteRandoms n gen =
    let (value :: a, newGen :: g) = random gen
        (restOfList :: [a], finalGen :: g) = finiteRandoms (n-1) newGen
    in  (value:restOfList, finalGen)

Again, a recursive definition. We say that if we want 0 numbers, we just
return an empty list and the generator that was given to us. For any
other number of random values, we first get one random number and a new
generator. That will be the head. Then we say that the tail will be *n -
1* numbers generated with the new generator. Then we return the head and
the rest of the list joined and the final generator that we got from
getting the *n - 1* random numbers.

What if we want a random value in some sort of range? All the random
integers so far were outrageously big or small. What if we want to to
throw a die? Well, we use __`randomR`__ for that purpose. It has a type of
`randomR :: (RandomGen g, Random a) :: (a, a) -> g -> (a, g)`, meaning
that it's kind of like `random`, only it takes as its first parameter a
pair of values that set the lower and upper bounds and the final value
produced will be within those bounds.

In [174]:
randomR (1,6) (mkStdGen 359353)

(4,StdGen {unStdGen = SMGen 10995245518073353784 11826319359189470311})

In [175]:
randomR (1,6) (mkStdGen 35935335)

(5,StdGen {unStdGen = SMGen 16742454905583792488 1242096894807673913})

There's also `randomRs`, which produces a stream of random values within
our defined ranges. Check this out:

In [176]:
take 10 $ randomRs ('a','z') (mkStdGen 3) :: [Char]

"zkjowfddqk"

Nice, looks like a super secret password or something.

You may be asking yourself, what does this section have to do with I/O
anyway? We haven't done anything concerning I/O so far. Well, so far
we've always made our random number generator manually by making it with
some arbitrary integer. The problem is, if we do that in our real
programs, they will always return the same random numbers, which is no
good for us. That's why `System.Random` offers the __`getStdGen`__ I/O action,
which has a type of `IO StdGen`. When your program starts, it asks the
system for a good random number generator and stores that in a so called
global generator. `getStdGen` fetches you that global random generator
when you bind it to something.

Here's a simple program that generates a random string.

In [177]:
import System.Random

main = do
    gen <- getStdGen
    putStr $ take 20 (randomRs ('a','z') gen)

In [178]:
main

onjkcxttzgabftfcoamd

In [179]:
main

onjkcxttzgabftfcoamd

In [180]:
main

onjkcxttzgabftfcoamd

Be careful though, just performing `getStdGen` twice will ask the system
for the same global generator twice. If you do this:

In [181]:
import System.Random

main = do
    gen <- getStdGen
    putStrLn $ take 20 (randomRs ('a','z') gen)
    gen2 <- getStdGen
    putStr $ take 20 (randomRs ('a','z') gen2)

In [182]:
main

onjkcxttzgabftfcoamd
onjkcxttzgabftfcoamd

you will get the same string printed out twice! One way to get two
different strings of length 20 is to set up an infinite stream and then
take the first 20 characters and print them out in one line and then
take the second set of 20 characters and print them out in the second
line. For this, we can use the [`splitAt`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:splitAt) function from `Data.List`, which
splits a list at some index and returns a tuple that has the first part
as the first component and the second part as the second component.

In [183]:
import System.Random
import Data.List

main = do
    gen <- getStdGen
    let randomChars = randomRs ('a','z') gen
        (first20, rest) = splitAt 20 randomChars
        (second20, _) = splitAt 20 rest
    putStrLn first20
    putStr second20

Another way is to use the __`newStdGen`__ action, which splits our current
random generator into two generators. It updates the global random
generator with one of them and encapsulates the other as its result.

In [184]:
import System.Random

main = do
    gen <- getStdGen
    putStrLn $ take 20 (randomRs ('a','z') gen)
    gen' <- newStdGen
    putStr $ take 20 (randomRs ('a','z') gen')

In [185]:
main

onjkcxttzgabftfcoamd
tmnvhlkvkduulvxexgvs

Not only do we get a new random generator when we bind `newStdGen` to
something, the global one gets updated as well, so if we do `getStdGen`
again and bind it to something, we'll get a generator that's not the
same as `gen`.

Here's a little program that will make the user guess which number it's
thinking of.

In [186]:
import System.Random
import Control.Monad(when)

main = do
    gen <- getStdGen
    askForNumber gen

askForNumber :: StdGen -> IO ()
askForNumber gen = do
    let (randNumber, newGen) = randomR (1,10) gen :: (Int, StdGen)
    putStrLn "Which number in the range from 1 to 10 am I thinking of? "
    numberString <- getLine
    when (not $ null numberString) $ do
        let number = read numberString
        if randNumber == number
            then putStrLn "You are correct!"
            else putStrLn $ "Sorry, it was " ++ show randNumber
        askForNumber newGen

Line 13: Use unless
Found:
when (not $ null numberString)
Why not:
unless (null numberString)

<img src="img/jackofdiamonds.png" title="jack of diamonds" style="background-color:white;float:left;margin-right:2em;" />

We make a function `askForNumber`, which takes a random number generator
and returns an I/O action that will prompt the user for a number and
tell him if he guessed it right. In that function, we first generate a
random number and a new generator based on the generator that we got as
a parameter and call them `randNumber` and `newGen`. Let's say that the
number generated was `7`. Then we tell the user to guess which number
we're thinking of. We perform [`getLine`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:getLine) and bind its result to
`numberString`. When the user enters `7`, `numberString` becomes `"7"`. Next, we
use [`when`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:when) to check if the string the user entered is an empty string. If
it is, an empty I/O action of `return ()` is performed, which effectively
ends the program. If it isn't, the action consisting of that *do* block
right there gets performed. We use [`read`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:read) on `numberString` to convert it to
a `number`, so number is now `7`.

> __Excuse me!__ If the user gives us some input here that [`read`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:read) can't read
> (like `"haha"`), our program will crash with an ugly error message. If you
> don't want your program to crash on erroneous input, use __[`reads`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:reads)__, which
> returns an empty list when it fails to read a string. When it succeeds,
> it returns a singleton list with a tuple that has our desired value as
> one component and a string with what it didn't consume as the other.

We check if the number that we entered is equal to the one generated
randomly and give the user the appropriate message. And then we call
`askForNumber` recursively, only this time with the new generator that we
got, which gives us an I/O action that's just like the one we performed,
only it depends on a different generator and we perform it.

`main` consists of just getting a random generator from the system and
calling `askForNumber` with it to get the initial action.

Here's our program in action!

In [187]:
withStdin (unlines ["4", "10", "2", "5"]) $ catchPrint main

Which number in the range from 1 to 10 am I thinking of? 
Sorry, it was 9
Which number in the range from 1 to 10 am I thinking of? 
Sorry, it was 2
Which number in the range from 1 to 10 am I thinking of? 
Sorry, it was 7
Which number in the range from 1 to 10 am I thinking of? 
Sorry, it was 3
Which number in the range from 1 to 10 am I thinking of? 
<stdin>: hGetLine: end of file

Another way to make this same program is like this:

In [188]:
import System.Random
import Control.Monad(when)

main = do
    gen <- getStdGen
    let (randNumber, _) = randomR (1,10) gen :: (Int, StdGen)
    putStr "Which number in the range from 1 to 10 am I thinking of? "
    numberString <- getLine
    when (not $ null numberString) $ do
        let number = read numberString
        if randNumber == number
            then putStrLn "You are correct!"
            else putStrLn $ "Sorry, it was " ++ show randNumber
        newStdGen
        main

Line 9: Use unless
Found:
when (not $ null numberString)
Why not:
unless (null numberString)

It's very similar to the previous version, only instead of making a
function that takes a generator and then calls itself recursively with
the new updated generator, we do all the work in `main`. After telling the
user whether they were correct in their guess or not, we update the
global generator and then call `main` again. Both approaches are valid but
I like the first one more since it does less stuff in `main` and also
provides us with a function that we can reuse easily.

Bytestrings
-----------

<img src="img/chainchomp.png" title="like normal string, only they byte ... what a pedestrian pun this is" style="background-color:white;float:right;margin-left:2em;"/>

Lists are a cool and useful data structure. So far, we've used them
pretty much everywhere. There are a multitude of functions that operate
on them and Haskell's laziness allows us to exchange the for and while
loops of other languages for filtering and mapping over lists, because
evaluation will only happen once it really needs to, so things like
infinite lists (and even infinite lists of infinite lists!) are no
problem for us. That's why lists can also be used to represent streams,
either when reading from the standard input or when reading from files.
We can just open a file and read it as a string, even though it will
only be accessed when the need arises.

However, processing files as strings has one drawback: it tends to be
slow. As you know, [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String) is a type synonym for `[Char]`. [`Char`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Char)s don't have
a fixed size, because it takes several bytes to represent a character
from, say, Unicode. Furthermore, lists are really lazy. If you have a
list like `[1,2,3,4]`, it will be evaluated only when completely
necessary. So the whole list is sort of a promise of a list. Remember
that `[1,2,3,4]` is syntactic sugar for `1:2:3:4:[]`. When the first element
of the list is forcibly evaluated (say by printing it), the rest of the
list `2:3:4:[]` is still just a promise of a list, and so on. So you can
think of lists as promises that the next element will be delivered once
it really has to and along with it, the promise of the element after it.
It doesn't take a big mental leap to conclude that processing a simple
list of numbers as a series of promises might not be the most efficient
thing in the world.

That overhead doesn't bother us so much most of the time, but it turns
out to be a liability when reading big files and manipulating them.
That's why Haskell has *bytestrings*. Bytestrings are sort of like
lists, only each element is one byte (or 8 bits) in size. The way they
handle laziness is also different.

Bytestrings come in two flavors: strict and lazy ones. Strict
bytestrings reside in `Data.ByteString` and they do away with the laziness
completely. There are no promises involved; a strict bytestring
represents a series of bytes in an array. You can't have things like
infinite strict bytestrings. If you evaluate the first byte of a strict
bytestring, you have to evaluate it whole. The upside is that there's
less overhead because there are no thunks (the technical term for
*promise*) involved. The downside is that they're likely to fill your
memory up faster because they're read into memory at once.

The other variety of bytestrings resides in `Data.ByteString.Lazy`.
They're lazy, but not quite as lazy as lists. Like we said before, there
are as many thunks in a list as there are elements. That's what makes
them kind of slow for some purposes. Lazy bytestrings take a different
approach — they are stored in chunks (not to be confused with thunks!),
each chunk has a size of 64K. So if you evaluate a byte in a lazy
bytestring (by printing it or something), the first 64K will be
evaluated. After that, it's just a promise for the rest of the chunks.
Lazy bytestrings are kind of like lists of strict bytestrings with a
size of 64K. When you process a file with lazy bytestrings, it will be
read chunk by chunk. This is cool because it won't cause the memory
usage to skyrocket and the 64K probably fits neatly into your CPU's L2
cache.

If you look through the
[documentation](http://hackage.haskell.org/package/bytestring/docs/Data-ByteString-Lazy.html)
for `Data.ByteString.Lazy`, you'll see that it has a lot of functions that
have the same names as the ones from `Data.List`, only the type signatures
have `ByteString` instead of `[a]` and `Word8` instead of `a` in them. The
functions with the same names mostly act the same as the ones that work
on lists. Because the names are the same, we're going to do a qualified
import in a script and then load that script into GHCI to play with
bytestrings.

In [189]:
import qualified Data.ByteString.Lazy as B
import qualified Data.ByteString as S

`B` has lazy bytestring types and functions, whereas [`S`](https://hackage.haskell.org/package/base/docs/GHC-Generics.html#t:S) has strict ones.
We'll mostly be using the lazy version.

The function __`pack`__ has the type signature `pack :: [Word8] -> ByteString`.
What that means is that it takes a list of bytes of type `Word8` and
returns a `ByteString`. You can think of it as taking a list, which is
lazy, and making it less lazy, so that it's lazy only at 64K intervals.

What's the deal with that `Word8` type? Well, it's like [`Int`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Int), only that it
has a much smaller range, namely 0-255. It represents an 8-bit number.
And just like [`Int`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Int), it's in the [`Num`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Num) typeclass. For instance, we know that
the value `5` is polymorphic in that it can act like any numeral type.
Well, it can also take the type of `Word8`.

In [190]:
B.pack [99,97,110]

"can"

In [191]:
B.pack [98..120]

"bcdefghijklmnopqrstuvwx"

As you can see, you usually don't have to worry about the `Word8` too
much, because the type system can makes the numbers choose that type. If
you try to use a big number, like `336` as a `Word8`, it will just wrap
around to `80`.

We packed only a handful of values into a `ByteString`, so they fit inside
one chunk. The `Empty` is like the `[]` for lists.

__`unpack`__ is the inverse function of `pack`. It takes a bytestring and turns
it into a list of bytes.

__`fromChunks`__ takes a list of strict bytestrings and converts it to a lazy
bytestring. __`toChunks`__ takes a lazy bytestring and converts it to a list
of strict ones.

In [192]:
B.fromChunks [S.pack [40,41,42], S.pack [43,44,45], S.pack [46,47,48]]

"()*+,-./0"

This is good if you have a lot of small strict bytestrings and you want
to process them efficiently without joining them into one big strict
bytestring in memory first.

The bytestring version of `:` is called __[`cons`](https://hackage.haskell.org/package/base/docs/Data-List-NonEmpty.html#v:cons)__ It takes a byte and a
bytestring and puts the byte at the beginning. It's lazy though, so it
will make a new chunk even if the first chunk in the bytestring isn't
full. That's why it's better to use the strict version of [`cons`](https://hackage.haskell.org/package/base/docs/Data-List-NonEmpty.html#v:cons), __`cons'`__ if
you're going to be inserting a lot of bytes at the beginning of a
bytestring.

In [193]:
B.cons 85 $ B.pack [80,81,82,84]

"UPQRT"

In [194]:
B.cons' 85 $ B.pack [80,81,82,84]

"UPQRT"

In [195]:
foldr B.cons B.empty [50..60]

"23456789:;<"

In [196]:
foldr B.cons' B.empty [50..60]

"23456789:;<"

As you can see __[`empty`](https://hackage.haskell.org/package/base/docs/Control-Applicative.html#v:empty)__ makes an empty bytestring. See the difference
between [`cons`](https://hackage.haskell.org/package/base/docs/Data-List-NonEmpty.html#v:cons) and `cons'`? With the [`foldr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:foldr), we started with an empty
bytestring and then went over the list of numbers from the right, adding
each number to the beginning of the bytestring. When we used [`cons`](https://hackage.haskell.org/package/base/docs/Data-List-NonEmpty.html#v:cons), we
ended up with one chunk for every byte, which kind of defeats the
purpose.

Otherwise, the bytestring modules have a load of functions that are
analogous to those in `Data.List`, including, but not limited to, [`head`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:head),
[`tail`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:tail), [`init`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:init), [`null`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:null), [`length`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:length), [`map`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:map), [`reverse`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:reverse), [`foldl`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:foldl), [`foldr`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:foldr), [`concat`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:concat), [`takeWhile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:takeWhile),
[`filter`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:filter), etc.

It also has functions that have the same name and behave the same as
some functions found in `System.IO`, only [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String)s are replaced with
`ByteString`s. For instance, the [`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile) function in `System.IO` has a type
of `readFile :: FilePath -> IO String`, while the __[`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile)__ from the
bytestring modules has a type of `readFile :: FilePath -> IO ByteString`.
Watch out, if you're using strict bytestrings and you attempt to read a
file, it will read it into memory at once! With lazy bytestrings, it
will read it into neat chunks.

Let's make a simple program that takes two filenames as command-line
arguments and copies the first file into the second file. Note that
`System.Directory` already has a function called `copyFile`, but we're going
to implement our own file copying function and program anyway.

In [197]:
import System.Environment
import qualified Data.ByteString.Lazy as B

main = do
    (fileName1:fileName2:_) <- getArgs
    copyFile fileName1 fileName2

copyFile :: FilePath -> FilePath -> IO ()
copyFile source dest = do
    contents <- B.readFile source
    B.writeFile dest contents

We make our own function that takes two [`FilePath`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:FilePath)s (remember, [`FilePath`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:FilePath) is
just a synonym for [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String)) and returns an I/O action that will copy one
file into another using bytestring. In the `main` function, we just get
the arguments and call our function with them to get the I/O action,
which is then performed.

In [198]:
withArgs ["/tmp/haiku.txt", "/tmp/haiku_mou_iku.txt"] main

Notice that a program that doesn't use bytestrings could look just like
this, the only difference is that we used `B.readFile` and `B.writeFile`
instead of [`readFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:readFile) and [`writeFile`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:writeFile). Many times, you can convert a program
that uses normal strings to a program that uses bytestrings by just
doing the necessary imports and then putting the qualified module names
in front of some functions. Sometimes, you have to convert functions
that you wrote to work on strings so that they work on bytestrings, but
that's not hard.

Whenever you need better performance in a program that reads a lot of
data into strings, give bytestrings a try, chances are you'll get some
good performance boosts with very little effort on your part. I usually
write programs by using normal strings and then convert them to use
bytestrings if the performance is not satisfactory.

Exceptions
----------

<img src="img/timber.png" title="timberr!!!!" style="background-color:white;float:left;margin-right:2em;" />

All languages have procedures, functions, and pieces of code that might
fail in some way. That's just a fact of life. Different languages have
different ways of handling those failures. In C, we usually use some
abnormal return value (like `-1` or a null pointer) to indicate that what
a function returned shouldn't be treated like a normal value. Java and
C\#, on the other hand, tend to use exceptions to handle failure. When
an exception is thrown, the control flow jumps to some code that we've
defined that does some cleanup and then maybe re-throws the exception so
that some other error handling code can take care of some other stuff.

Haskell has a very good type system. Algebraic data types allow for
types like [`Maybe`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Maybe) and [`Either`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Either) and we can use values of those types to
represent results that may be there or not. In C, returning, say, `-1` on
failure is completely a matter of convention. It only has special
meaning to humans. If we're not careful, we might treat these abnormal
values as ordinary ones and then they can cause havoc and dismay in our
code. Haskell's type system gives us some much-needed safety in that
aspect. A function `a -> Maybe b` clearly indicates that it it may
produce a `b` wrapped in [`Just`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:Just) or that it may return [`Nothing`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:Nothing). The type is
different from just plain `a -> b` and if we try to use those two
functions interchangeably, the compiler will complain at us.

Despite having expressive types that support failed computations,
Haskell still has support for exceptions, because they make more sense
in I/O contexts. A lot of things can go wrong when dealing with the
outside world because it is so unreliable. For instance, when opening a
file, a bunch of things can go wrong. The file might be locked, it might
not be there at all or the hard disk drive or something might not be
there at all. So it's good to be able to jump to some error handling
part of our code when such an error occurs.

Okay, so I/O code (i.e. impure code) can throw exceptions. It makes
sense. But what about pure code? Well, it can throw exceptions too.
Think about the [`div`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:div) and [`head`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:head) functions. They have types of
`(Integral a) => a -> a -> a` and `[a] -> a`, respectively. No [`Maybe`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Maybe) or [`Either`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Either) in
their return type and yet they can both fail! [`div`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:div) explodes in your face
if you try to divide by zero and [`head`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:head) throws a tantrum when you give it
an empty list.

In [199]:
4 `div` 0

: 

In [200]:
head []

: 

<img src="img/police.png" title="Stop right there, criminal scum! Nobody breaks the law on my watch! Now pay your fine or it's off to jail." style="background-color:white;float:left;margin-right:2em;" />

Pure code can throw exceptions, but they can only be caught in the
I/O part of our code (when we're inside a *do* block that goes into
`main`). That's because you don't know when (or if) anything will be
evaluated in pure code, because it is lazy and doesn't have a
well-defined order of execution, whereas I/O code does.

Earlier, we talked about how we should spend as little time as possible
in the I/O part of our program. The logic of our program should reside
mostly within our pure functions, because their results are dependant
only on the parameters that the functions are called with. When dealing
with pure functions, you only have to think about what a function
returns, because it can't do anything else. This makes your life easier.
Even though doing some logic in I/O is necessary (like opening files and
the like), it should preferably be kept to a minimum. Pure functions are
lazy by default, which means that we don't know when they will be
evaluated and that it really shouldn't matter. However, once pure
functions start throwing exceptions, it matters when they are evaluated.
That's why we can only catch exceptions thrown from pure functions in
the I/O part of our code. And that's bad, because we want to keep the
I/O part as small as possible. However, if we don't catch them in the
I/O part of our code, our program crashes. The solution? Don't mix
exceptions and pure code. Take advantage of Haskell's powerful type
system and use types like [`Either`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Either) and [`Maybe`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Maybe) to represent results that may
have failed.

That's why we'll just be looking at how to use I/O exceptions for now.
I/O exceptions are exceptions that are caused when something goes wrong
while we are communicating with the outside world in an I/O action
that's part of `main`. For example, we can try opening a file and then it
turns out that the file has been deleted or something. Take a look at
this program that opens a file whose name is given to it as a command
line argument and tells us how many lines the file has.

In [201]:
import System.Environment
import System.IO

main = do (fileName:_) <- getArgs
          contents <- readFile fileName
          putStrLn $ "The file has " ++ show (length (lines contents)) ++ " lines!"

A very simple program. We perform the [`getArgs`](https://hackage.haskell.org/package/base/docs/System-Environment.html#v:getArgs) I/O action and bind the
first string in the list that it yields to `fileName`. Then we call the
`contents` of the file with that name contents. Lastly, we apply [`lines`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:lines) to
those contents to get a list of lines and then we get the length of that
list and give it to [`show`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:show) to get a string representation of that number.
It works as expected, but what happens when we give it the name of a
file that doesn't exist?

In [202]:
withArgs ["i_dont_exist.txt"] main

: 

Aha, we get an error from GHC, telling us that the file does not exist.
Our program crashes. What if we wanted to print out a nicer message if
the file doesn't exist? One way to do that is to check if the file
exists before trying to open it by using the __`doesFileExist`__ function from
`System.Directory`.

In [203]:
import System.Environment
import System.IO
import System.Directory

main = do (fileName:_) <- getArgs
          fileExists <- doesFileExist fileName
          if fileExists
              then do contents <- readFile fileName
                      putStrLn $ "The file has " ++ show (length (lines contents)) ++ " lines!"
              else do putStrLn "The file doesn't exist!"

We did `fileExists <- doesFileExist fileName` because `doesFileExist` has a
type of `doesFileExist :: FilePath -> IO Bool`, which means that it
returns an I/O action that has as its result a boolean value which tells
us if the file exists. We can't just use `doesFileExist` in an *if*
expression directly.

Another solution here would be to use exceptions. It's perfectly
acceptable to use them in this context. A file not existing is an
exception that arises from I/O, so catching it in I/O is fine and dandy.

To deal with this by using exceptions, we're going to take advantage of
the __[`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch)__ function from `System.IO.Error`. Its type is
`catch :: IO a -> (IOError -> IO a) -> IO a`.
It takes two parameters. The first one is
an I/O action. For instance, it could be an I/O action that tries to
open a file. The second one is the so-called handler. If the first I/O
action passed to [`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch) throws an I/O exception, that exception gets
passed to the handler, which then decides what to do. So the final
result is an I/O action that will either act the same as the first
parameter or it will do what the handler tells it if the first I/O
action throws an exception.

<img src="img/puppy.png" title="non sequitor" style="background-color:white;float:right;margin-left:2em;" />

If you're familiar with *try-catch* blocks in languages like Java or
Python, the [`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch) function is similar to them. The first parameter is
the thing to try, kind of like the stuff in the *try* block in other,
imperative languages. The second parameter is the handler that takes an
exception, just like most *catch* blocks take exceptions that you can
then examine to see what happened. The handler is invoked if an
exception is thrown.

The handler takes a value of type [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError), which is a value that
signifies that an I/O exception occurred. It also carries information
regarding the type of the exception that was thrown. How this type is
implemented depends on the implementation of the language itself, which
means that we can't inspect values of the type [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) by pattern
matching against them, just like we can't pattern match against values
of type `IO <something>`. We can use a bunch of useful predicates to find
out stuff about values of type [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) as we'll learn in a second.

So let's put our new friend [`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch) to use!

In [204]:
import System.Environment
import System.IO
import System.IO.Error
import Control.Exception

main = toTry `catch` handler

toTry :: IO ()
toTry = do (fileName:_) <- getArgs
           contents <- readFile fileName
           putStrLn $ "The file has " ++ show (length (lines contents)) ++ " lines!"

handler :: IOError -> IO ()
handler e = putStrLn "Whoops, had some trouble!"

First of all, you'll see that put backticks around it so that we can use
it as an infix function, because it takes two parameters. Using it as an
infix function makes it more readable. So ``toTry `catch` handler`` is the
same as `catch toTry handler`, which fits well with its type. `toTry` is the
I/O action that we try to carry out and `handler` is the function that
takes an [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) and returns an action to be carried out in case of an
exception.

Let's give this a go:

In [205]:
withArgs ["/tmp/haiku.txt"] main

The file has 3 lines!

In [206]:
withArgs ["i_dont_exists.txt"] main

Whoops, had some trouble!

In the handler, we didn't check to see what kind of [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) we got. We
just say `"Whoops, had some trouble!"` for any kind of error. Just
catching all types of exceptions in one handler is bad practice in
Haskell just like it is in most other languages. What if some other
exception happens that we don't want to catch, like us interrupting the
program or something? That's why we're going to do the same thing that's
usually done in other languages as well: we'll check to see what kind of
exception we got. If it's the kind of exception we're waiting to catch,
we do our stuff. If it's not, we throw that exception back into the
wild. Let's modify our program to catch only the exceptions caused by a
file not existing.

In [207]:
import System.Environment
import System.IO
import System.IO.Error

main = toTry `catch` handler

toTry :: IO ()
toTry = do (fileName:_) <- getArgs
           contents <- readFile fileName
           putStrLn $ "The file has " ++ show (length (lines contents)) ++ " lines!"

handler :: IOError -> IO ()
handler e
    | isDoesNotExistError e = putStrLn "The file doesn't exist!"
    | otherwise = ioError e

Everything stays the same except the handler, which we modified to only
catch a certain group of I/O exceptions. Here we used two new functions
from `System.IO.Error` — __[`isDoesNotExistError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isDoesNotExistError)__ and __[`ioError`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:ioError)__.
[`isDoesNotExistError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isDoesNotExistError) is a predicate over `IOErrors`, which means that it's
a function that takes an [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) and returns a [`True`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:True) or [`False`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:False), meaning it
has a type of `isDoesNotExistError :: IOError -> Bool`. We use it on the
exception that gets passed to our handler to see if it's an error caused
by a file not existing. We use
[guard](http://learnyouahaskell.com/syntax-in-functions#guards-guards) syntax here, but we could
have also used an *if else*. If it's not caused by a file not existing,
we re-throw the exception that was passed by the handler with the
[`ioError`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:ioError) function. It has a type of `ioError :: IOException -> IO a`, so
it takes an [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) and produces an I/O action that will throw it. The
I/O action has a type of `IO a`, because it never actually yields a
result, so it can act as [`IO`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IO) *`anything`*.

So the exception thrown in the `toTry` I/O action that we glued together
with a *do* block isn't caused by a file existing,
``toTry `catch` handler`` will catch that and then re-throw it. Pretty cool, huh?

There are several predicates that act on [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) and if a guard doesn't
evaluate to [`True`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:True), evaluation falls through to the next guard. The
predicates that act on [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) are:

-   __[`isAlreadyExistsError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isAlreadyExistsError)__
-   __[`isDoesNotExistError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isDoesNotExistError)__
-   __[`isAlreadyInUseError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isAlreadyInUseError)__
-   __[`isFullError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isFullError)__
-   __[`isEOFError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isEOFError)__
-   __[`isIllegalOperation`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isIllegalOperation)__
-   __[`isPermissionError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isPermissionError)__
-   __[`isUserError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isUserError)__

Most of these are pretty self-explanatory. __[`isUserError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isUserError)__ evaluates to [`True`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:True)
when we use the function [`userError`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:userError) to make the exception, which is used
for making exceptions from our code and equipping them with a string.
For instance, you can do
`ioError $ userError "remote computer unplugged!"`,
although it's prefered you use types like [`Either`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Either) and [`Maybe`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Maybe)
to express possible failure instead of throwing exceptions yourself with
[`userError`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:userError).

So you could have a handler that looks something like this:

<pre><code>handler :: IOError -> IO ()
handler e
    | isDoesNotExistError e = putStrLn "The file doesn't exist!"
    | isFullError e = freeSomeSpace
    | isIllegalOperation e = notifyCops
    | otherwise = ioError e</code></pre>

Where `notifyCops` and `freeSomeSpace` are some I/O actions that you define.
Be sure to re-throw exceptions if they don't match any of your criteria,
otherwise you're causing your program to fail silently in some cases
where it shouldn't.

`System.IO.Error` also exports functions that enable us to ask our
exceptions for some attributes, like what the handle of the file that
caused the error is, or what the filename is. These start with `ioe` and
you can see a [full list of
them](http://www.haskell.org/ghc/docs/6.10.1/html/libraries/base/System-IO-Error.html#3)
in the documentation. Say we want to print the filename that caused our
error. We can't print the `fileName` that we got from [`getArgs`](https://hackage.haskell.org/package/base/docs/System-Environment.html#v:getArgs), because
only the [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) is passed to the handler and the handler doesn't know
about anything else. A function depends only on the parameters it was
called with. That's why we can use the __[`ioeGetFileName`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:ioeGetFileName)__ function, which
has a type of `ioeGetFileName :: IOError -> Maybe FilePath`. It takes an
[`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError) as a parameter and maybe returns a [`FilePath`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:FilePath) (which is just a
type synonym for [`String`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:String), remember, so it's kind of the same thing).
Basically, what it does is it extracts the file path from the [`IOError`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IOError),
if it can. Let's modify our program to print out the file path that's
responsible for the exception occurring.

In [208]:
import System.Environment
import System.IO
import System.IO.Error

main = toTry `catch` handler

toTry :: IO ()
toTry = do (fileName:_) <- getArgs
           contents <- readFile fileName
           putStrLn $ "The file has " ++ show (length (lines contents)) ++ " lines!"

handler :: IOError -> IO ()
handler e
    | isDoesNotExistError e =
        case ioeGetFileName e of Just path -> putStrLn $ "Whoops! File does not exist at: " ++ path
                                 Nothing -> putStrLn "Whoops! File does not exist at unknown location!"
    | otherwise = ioError e

In [209]:
withArgs ["./i_dont_exist.txt"] main

Whoops! File does not exist at: ./i_dont_exist.txt

In the guard where [`isDoesNotExistError`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:isDoesNotExistError) is [`True`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:True), we used a *case*
expression to call [`ioeGetFileName`](https://hackage.haskell.org/package/base/docs/System-IO-Error.html#v:ioeGetFileName) with `e` and then pattern match against
the [`Maybe`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Maybe) value that it returned. Using *case* expressions is commonly
used when you want to pattern match against something without bringing
in a new function.

You don't have to use one handler to [`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch) exceptions in your whole I/O
part. You can just cover certain parts of your I/O code with [`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch) or
you can cover several of them with [`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch) and use different handlers for
them, like so:

<pre><code>main = do toTry `catch` handler1
          thenTryThis `catch` handler2
          launchRockets</code></pre>

Here, `toTry` uses `handler1` as the handler and `thenTryThis` uses `handler2`.
`launchRockets` isn't a parameter to [`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch), so whichever exceptions it
might throw will likely crash our program, unless `launchRockets` uses
[`catch`](https://hackage.haskell.org/package/base/docs/Control-Exception.html#v:catch) internally to handle its own exceptions. Of course `toTry`,
`thenTryThis` and `launchRockets` are I/O actions that have been glued
together using *do* syntax and hypothetically defined somewhere else.
This is kind of similar to *try-catch* blocks of other languages, where
you can surround your whole program in a single *try-catch* or you can
use a more fine-grained approach and use different ones in different
parts of your code to control what kind of error handling happens where.

Now you know how to deal with I/O exceptions! Throwing exceptions from
pure code and dealing with them hasn't been covered here, mainly
because, like we said, Haskell offers much better ways to indicate
errors than reverting to I/O to catch them. Even when glueing together
I/O actions that might fail, I prefer to have their type be something
like `IO (Either a b)`, meaning that they're normal I/O actions but the
result that they yield when performed is of type `Either a b`, meaning
it's either `Left a` or `Right b`.